# Transit Service Areas - definition and summaries

Notebook is an implementation of https://github.com/BayAreaMetro/bayarea_urbansim/blob/main/scripts/proximity2transit.py. It will be turned into a standalone script.

In words, the process is going from a point layer of transit stops with information on size and headways and build status. Those are to be buffered, and a hierarchy is active, favoring rail over non-rail, and shorter headways over longer headways. Service areas a built by overlays, and parcels are classified by the service area they fall within. Finally model runs are summarized by these service areas, and other attributes.

# Paraphernalia

In [2]:
import os
import sys
import shutil
import time
import traceback
import logging
import datetime
from pathlib import Path

import pandas as pd
import geopandas as gpd
import fiona

import seaborn as sns
import matplotlib.pyplot as plt


/Users/aolsen/opt/anaconda3/envs/geo_env_2/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_17729/688566035.py:11: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Sha

In [3]:
# DVUTILS_LOCAL_CLONE_PATH = "/Users/aolsen/Documents/GitHub/dvutils"
# sys.path.insert(0, DVUTILS_LOCAL_CLONE_PATH)
# from utils_io import *
user = os.getlogin()
user

'aolsen'

In [4]:
# # gdb_path = '/Volumes/Data/Models/Scratch/AO/workspace_2024_0315_1017.gdb'
# gdb_path = '/Volumes/Data/Models/Data/GIS layers/JobsHousingTransitProximity/workspace_2020_1008_2343.gdb'

# #fiona.listlayers(gdb_path)
# v2020_fbp_classes = gpd.read_file(gdb_path,layer='trn_bp_cat5')
# v2020_cur_classes = gpd.read_file(gdb_path,layer='trn_cur_cat5')
# v2020_np_classes = gpd.read_file(gdb_path,layer='trn_np_cat5')

In [5]:
# v2020_fbp_classes = gpd.read_file(gdb_path,layer='trn_bp_cat5')
# v2020_cur_classes = gpd.read_file(gdb_path,layer='trn_cur_cat5')
# v2020_np_classes = gpd.read_file(gdb_path,layer='trn_np_cat5')

In [6]:
# fiona.listlayers(gdb_path)
# v2020_transit_current = gpd.read_file(gdb_path,layer='transit_current')
# v2020_transit_potential = gpd.read_file(gdb_path,layer='transit_potential')
# v2020_transit_cur_major = gpd.read_file(gdb_path,layer='trn_cur_major')

## Paths
### AGOL URLs, and local versions until we get utils working

In [7]:
ANALYSIS_CRS = 'EPSG:26910'

In [8]:
#WORKING_DIR = os.path.abspath(".")
WORKING_DIR = '/Volumes/Data/Models/Data/GIS layers/JobsHousingTransitProximity/update_2024'

In [9]:
MTC_ONLINE_TRANSIT_URL = 'https://services3.arcgis.com/i2dkYWmb4wHvYPda/arcgis/rest/services/transitstops_existing_planned_2021/FeatureServer/0'

MTC_ONLINE_BACOUNTY_URL = 'https://services3.arcgis.com/i2dkYWmb4wHvYPda/ArcGIS/rest/services/region_county/FeatureServer/0'

MTC_ONLINE_TAZ_URL = 'https://services3.arcgis.com/i2dkYWmb4wHvYPda/ArcGIS/rest/services/transportation_analysis_zones_1454/FeatureServer/0'

#MTC_ONLINE_TRACT_URL = 'https://services3.arcgis.com/i2dkYWmb4wHvYPda/ArcGIS/rest/services/Bay_Area_Census_Tracts_2010/FeatureServer/0'
MTC_ONLINE_TRACT_URL = 'https://services3.arcgis.com/i2dkYWmb4wHvYPda/ArcGIS/rest/services/cocs_ACS2014_2018/FeatureServer/0'

In [10]:
tract_epc21_path = '/Volumes/Data/Models/Data/Equity Priority Communities/PBA50/EPCs_ACS2018_tbl.csv'
tract_epc24_path = '/Volumes/Data/Models/Data/Equity Priority Communities/PBA50Plus/epc_acs2022.csv'

tract_epc_path = '/Volumes/Data/Models/Data/GIS layers/JobsHousingTransitProximity/update_2024/inputs/proximity/communities_of_concern_2020_acs2018_-5728172521147435275.gpkg'

county_path = '/Volumes/Data/Models/Data/GIS layers/JobsHousingTransitProximity/update_2024/inputs/proximity/region_county_-6301486166122500334.gpkg'
transit_path = '/Volumes/Data/Models/Data/GIS layers/JobsHousingTransitProximity/update_2024/inputs/proximity/transitstops_existing_planned_2021_2777787894694541529.gpkg'
taz_path = '/Volumes/Data/Models/Data/GIS layers/JobsHousingTransitProximity/update_2024/inputs/proximity/transportation_analysis_zones_1454_5961535026220395633.gpkg'

In [122]:
tract_hra_2023_path = '/Volumes/Data/Models/Data/HCD/final_2023_public.xlsx'
tract_hra_2023 = pd.read_excel(tract_hra_2023_path,'BayArea',dtype={'Census Tract':str}).rename(columns={'Census Tract':'geoid','Final Category':'hra_category'})
#.set_index('Tract FIPS Code')['Final Category']
tract_hra_2023.hra_category = tract_hra_2023.hra_category.fillna('Missing')


In [123]:
tract_hra_2019_path = '/Volumes/Data/Models/Data/HCD/final-opportunity-map-statewide-summary-table.xlsx'
tract_hra_2019 = pd.read_excel(tract_hra_2019_path, 'BayArea', dtype={
                               'Tract FIPS Code': str}).rename(columns={'Tract FIPS Code':'geoid','Final Category':'hra_category'})
#.set_index('Tract FIPS Code')['Final Category']
tract_hra_2019.hra_category = tract_hra_2019.hra_category.fillna('Missing')


In [13]:
tract_epc21_file = pd.read_csv(tract_epc21_path)
tract_epc24_file = pd.read_csv(tract_epc24_path)

In [14]:
county_file = gpd.read_file(county_path)
county_file = county_file.to_crs(ANALYSIS_CRS)

In [15]:
transit_file = gpd.read_file(transit_path)
transit_file = transit_file.to_crs(ANALYSIS_CRS)

In [16]:
taz_file = gpd.read_file(taz_path)
taz_file = taz_file.to_crs(ANALYSIS_CRS)

In [17]:
# Enforce 11-character tract GEOID
tract_epc21_file['geoid10'] = tract_epc21_file.geoid.map(lambda x: f'{x:011d}')
tract_epc24_file['geoid20'] = tract_epc24_file.tract_geoid.map(lambda x: f'{x:011d}')

In [78]:
### Tracts
tract_2020_file = '/Volumes/Data/Models/Data/GIS layers/Census/2020/censustracts_bayarea_2020_v2.shp'
tract_2010_file = '/Volumes/Data/Models/Data/GIS layers/Census/2010/tracts_bayarea_2010_geoid.shp'

In [79]:
tract_2010 = gpd.read_file(tract_2010_file,engine='pyogrio')
tract_2020 = gpd.read_file(tract_2020_file,engine='pyogrio')

In [ ]:
# confirming vintages for EPC files
def confirm_tract_vintage(fl,tr, fl_col,tr_col):
    output = list(set(fl[fl_col])-set(tr[tr_col]))
    print('Unmatched tracts: ',len(output))
    return output

In [110]:
# check that epc21 tracts are well matched in census 2010 vintage
confirm_tract_vintage(fl=tract_epc21_file, tr=tract_2010,fl_col='geoid10',tr_col='GEOID10')

Unmatched tracts:  5


['06041990100', '06075990100', '06075980401', '06001990000', '06097990100']

In [109]:
# check that epc24 tracts are well matched in census 2020 vintage
confirm_tract_vintage(fl=tract_epc24_file, tr=tract_2020,fl_col='geoid20',tr_col='GEOID')

Unmatched tracts:  6


['06041990100',
 '06075990100',
 '06075980401',
 '06001990000',
 '06097990100',
 '06081990100']

In [203]:
confirm_tract_vintage(fl=tract_hra_2019, tr=tract_2010,fl_col='geoid',tr_col='GEOID10')

Unmatched tracts:  5


['06041990100', '06075990100', '06075980401', '06001990000', '06097990100']

In [202]:
# this works nicely with tracts 2010, but not tracts 2020

confirm_tract_vintage(fl=tract_hra_2023, tr=tract_2010,fl_col='geoid',tr_col='GEOID10')

Unmatched tracts:  5


['06041990100', '06075990100', '06075980401', '06001990000', '06097990100']

### Crosswalks

In [18]:
# set up crosswalks for parcels to something else
parcel_x_taz_file = '/Volumes/Data/Models/urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/2020_08_17_parcel_to_taz1454sub.csv'
parcel_x_tract10_file = '/Volumes/Data/Models/urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/parcel_tract_crosswalk.csv'
parcel_x_tracts_file = '/Volumes/Data/Models/urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/p10_census.csv'
parcel_topo_file = '/Users/aolsen/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/Policies/Base zoning/inputs/p10_geo.feather'
taz_x_areatype_file = '/Volumes/Data/Models/Application/Model One/RTP2021/Blueprint/INPUT_DEVELOPMENT/metrics/metrics_FinalBlueprint/taz_urban_suburban.csv'

In [19]:
parcel_x_taz = pd.read_csv(parcel_x_taz_file,
                           usecols=['PARCEL_ID', 'ZONE_ID'])

In [20]:
parcel_x_tract10 = pd.read_csv(parcel_x_tract10_file,
                               usecols=[
                                   'parcel_id', 'zone_id', 'GEOID10'])

In [21]:
taz_x_areatype = pd.read_csv(taz_x_areatype_file,index_col=['TAZ1454']).area_type

In [22]:
parcel_x_tracts = pd.read_csv(parcel_x_tracts_file)

In [23]:
# load p10 parcels

p10_topofix = gpd.read_feather(parcel_topo_file)
p10_topofix['geom_pt'] = p10_topofix.representative_point()
p10_topofix_pt = p10_topofix.set_geometry('geom_pt')[['PARCEL_ID', 'geom_pt']]

In [24]:
region_bdry = gpd.GeoDataFrame(data={'No_Fixed_Route_Transit':[0]},geometry=county_file.dissolve()['geometry'])
region_bdry.index = region_bdry.index.set_names('rowid')

### BAUS paths

In [25]:
urbansim_run_location = f'/Users/{user}/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/'
# keep DBP for future
us_2050_DBP_Final = 'Draft Blueprint runs/Blueprint Plus Crossing (s23)/v1.7.1- FINAL DRAFT BLUEPRINT/run98'
us_2050_FBP_Final = 'Final Blueprint runs/Final Blueprint (s24)/BAUS v2.25 - FINAL VERSION/run182'
us_2050_NP_EIR = 'EIR runs/Baseline Large (s25) runs/NP_v8_FINAL/run314'
us_2050_ALT1_EIR = 'EIR runs/Alt1 (s26) runs/Alt1_v3_test_far_tiers_FINAL_EIR_ALT/run375'
us_2050_ALT2_EIR = 'EIR runs/Alt2 (s28) runs/Alt2_v1_FINAL_EIR_ALT/run374'

list_us_runid = [us_2050_NP_EIR, us_2050_ALT1_EIR, us_2050_ALT2_EIR,
                 us_2050_FBP_Final]  # , us_2050_FBP_Final, us_2050_DBP_Final]
list_us_runid

['EIR runs/Baseline Large (s25) runs/NP_v8_FINAL/run314',
 'EIR runs/Alt1 (s26) runs/Alt1_v3_test_far_tiers_FINAL_EIR_ALT/run375',
 'EIR runs/Alt2 (s28) runs/Alt2_v1_FINAL_EIR_ALT/run374',
 'Final Blueprint runs/Final Blueprint (s24)/BAUS v2.25 - FINAL VERSION/run182']

## Constants and mappings

In [26]:
KM_TO_MILES = 1.609344

# get the inverse
MILES_TO_KM = 1/KM_TO_MILES

In [27]:

def miles_to_m(miles):
    """Converts miles to meters.

    Args:
      miles: The distance in miles.

    Returns:
      The distance in meters.
    """
    return 1000 * miles / MILES_TO_KM

In [28]:
def get_combination_names(group):
    """Constructs a string representing combinations based on True values in a pandas GroupBy.

    Generates a string by joining column names where the corresponding value in the Series is True.

    This function is designed to be used with the `apply` method of a pandas GroupBy object.

    Args:
    group: A Series (from a pandas GroupBy) containing boolean values.

    Returns:
    A string with '-' joining column names that have True values, or an empty string if none.
    """

    combination_names = []
    for col_name, value in group.items():
        if value.any():
            # if there are any true values - keep the corresponding column name
            combination_names.append(col_name)
    return '-'.join(combination_names)

In [29]:
def get_combination_names_hierarchy(group):
    """Determines a hierarchical name for a group in a pandas groupby based on True values.

    Prioritizes the first column with all True values (out of four boolean transit stop buffer categories) 
    excluding 'No_Fixed_Route_Transit'.

    This function is designed to be used with the `apply` method of a pandas GroupBy object.

    Args:
    group: A Series (from a pandas GroupBy) containing boolean values for the four transit stop buffer categories.

    Returns:
    A string representing the hierarchical name, either a feature name with '_only' 
    suffix, 'none' if only 'No_Fixed_Route_Transit' is True, or the original group 
    if no single feature has all True values.
    """
    
    # We loop through values in the tuple of booleans for each group - ordering matters. So
    # we stop at the first case of a True value - the 
    # first one takes primacy - like major stops buffer
    for col, val in group.items():
        # print(col,val)
        if val.all():
            return f'{col}_only' if col!='No_Fixed_Route_Transit' else 'none'

    return group

In [30]:
def station_buffer(source_df, buf_dist):
    """Buffers a GeoDataFrame with a specified distance in miles.

    Args:
    source_df: A GeoDataFrame containing the stations to be buffered.
    buf_dist: The distance in miles for the buffer zone.

    Returns:
    A GeoDataFrame representing the stations with a buffer applied 
    and dissolved to remove duplicate geometries.

    Logs:
    The size of the dataframe and the buffer applied.
    """
    logger.info(f'\tBuffering a df of {len(source_df)} records with {buf_dist:.02f} miles')
    source_buf = source_df.buffer(miles_to_m(buf_dist))

    output = (gpd.GeoDataFrame(  # data=source_subset[passthrough_cols],
        geometry=source_buf.values)
        .reset_index()
        .rename(columns={'index': 'rowid'}))

    return output.dissolve()

# Create buffer for transit layer

## Set up logger

In [31]:
# create logger
NOW = datetime.datetime.now()
YDM = f'process_{NOW:%Y%m%d}'
LOG_FILE = Path(WORKING_DIR, "proximity2transit_{}.log".format(NOW))
logger = logging.getLogger(__name__)
logger.setLevel('DEBUG')

# console handler
ch = logging.StreamHandler()
ch.setLevel('INFO')
ch.setFormatter(logging.Formatter(
    '%(asctime)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p'))
logger.addHandler(ch)

# file handler
fh = logging.FileHandler(LOG_FILE, mode='w')
fh.setLevel('DEBUG')
fh.setFormatter(logging.Formatter(
    '%(asctime)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p'))
logger.addHandler(fh)
today_path = Path(WORKING_DIR,'outputs',YDM)
today_path.mkdir(exist_ok=True)

In [32]:
new_fgdb = Path(today_path,'file_geodatabase.gdb')

In [33]:
#shutil.copytree(Path(WORKING_DIR,'empty_fgdb.gdb'),new_fgdb)

## Step 0: Subset transit data to relevant universes

In [34]:
# we use status criteria OR major_stop here -
# following logic here: https://github.com/BayAreaMetro/petrale/blob/c4b96a98b291ada58e065375beccd5bf11e2da1b/scripts/proximity2transit.py#L128

# subset to existing, under construction, open or FBP
# this is really just everything

# | major_stop==1
qry_fbp = 'status.isin(["Under Construction","Open","Final Blueprint","Existing/Built"]) '

transit_fbp = transit_file.query(
    qry_fbp)

# subset to existing major stop, and under construction or open
# | (major_stop==1 & status!="Final Blueprint")
qry_np = 'status.isin(["Under Construction","Open","Existing/Built"]) '

transit_np = transit_file.query(
    qry_np)

# subset to existing transit infrastructure (per 2015 or so, pre-plan)

qry_current = ' status=="Existing/Built"'
#qry_current = 'major_stop==1 '
transit_current = transit_file.query(
    qry_current)

print(transit_np.shape)
print(transit_fbp.shape)
print(transit_current.shape)

(40237, 18)
(40577, 18)
(40232, 18)


In [35]:
def data_filtering(input_data, filter_criteria):
    """Filters a DataFrame of transit stops based on specified criteria.

    Args:
        input_data (pd.DataFrame): The DataFrame containing input data.
        filter_criteria (dict): A dictionary where keys are column names and values are tuples
            containing the comparison operator and the value to compare against.

    Returns:
        pd.DataFrame: The filtered DataFrame.
    """

    key_numerics = ['am_av_hdwy', 'pm_av_hdwy', 'major_stop']

    logger.info(
        '\tStarting length of the dataframe: {:,}'.format(len(input_data)))

    # for filter_criteria in list_filter_criteria:
    filter_components = []
    for key, operator_values in filter_criteria.items():
        filter_sub_components = []

        # skip non-criteria related keys in dict
        if not key in ['name', 'buffer']:
            # print(operator_values)
            for operator_value in operator_values:
                value = operator_value['value']
                operator = operator_value['operator']

                # Generate query expression list from components

                # if value is a string  - add quotes - otherwise assume to be numeric
                lst_conditions = [
                    f'{key}{operator}"{value}"' if isinstance(
                        value, str) else f"{key}{operator}{value}"
                ]

                # Generate query string from query list
                str_conditions = "".join(lst_conditions)
                filter_sub_components.append(str_conditions)

            # we are treating inner components as AND - so am_headway >10 and am_headway_15
            # but we are treating AM and PM headways as OR - a stop with 12 minutes am and 25 minutes pm
            # will be selected with the shorter headway
            str_partial_query = '({})'.format(
                " & ".join(filter_sub_components))
            filter_components.append(str_partial_query)

    # Concatenate component filters
    str_full_query = " | ".join(filter_components)
    logger.info(f'\tQuery string: {str_full_query}')

    # Create mask of matching records from component filters - to drop
    keep_mask = input_data.eval(str_full_query)

    if len(keep_mask.value_counts()) > 1:
        keep_summary = input_data[keep_mask][key_numerics].median()
        series_string = f"\tRecords Filtered: {'; '.join([f'{key}: {value:,.0f}' for key, value in keep_summary.items()])}"

#         logger.info(
#             f'Applying filter: `{str_full_query}`\n',
#             f'\Selecting {keep_mask.value_counts()[True]} records from dataframe\n',
#             #f'\tRecords lost:\n\t{input_data[keep_mask][key_numerics].sum()}',
#             series_string)

        # applying to source data
        input_data = input_data[keep_mask]
        logger.info(f'\tRecords after subsetting: {len(input_data):,.0f}')

    else:
        logger.info('Filters match no records. Keeping all records.')

    # Return resulting df
    return input_data

In [36]:
# transit_filtered = data_filtering(transit_current, filter_criteria_hdwy_11_to_15)
# transit_filtered

In [37]:
transit_buffers = {}

def create_multiple_transit_areas(transit_stop_gdf, criteria_list, slug='current', write_to_disk=False):

    logger.info(
        f"***Running create_transit_areas function for variant: {slug}***")
    logger.info(
        f"Transit Stop input geodataframe has shape: {transit_stop_gdf.shape}")

    # store filtered and buffered data
    buffered_stops = {}

    # store the names of the passed filter criteria - order matters to the hierarchy
    # and needs to be known to the user - that major stops will trump 15 min headways, will trum 30 min headways etc
    source_frames_ordered = []

    logger.info('***Looping through criteria list to subset transit stops***')

    # filter transit data, and in the same go, buffer the stops

    for filter_criteria in criteria_list:
        logger.info(f'Transit Station Subset: {filter_criteria["name"]}')

        # store filter condition name
        source_frames_ordered.append(filter_criteria["name"])
        transit_filtered = data_filtering(transit_stop_gdf, filter_criteria)

        # apply the filter
        transit_filtered_buffered = station_buffer(
            transit_filtered, filter_criteria['buffer'])

        # store filtered and buffered data in a dict
        buffered_stops[filter_criteria['name']] = transit_filtered_buffered

        transit_filtered.to_file(Path(today_path,'transit_filter_{}.gpkg'.format(filter_criteria["name"])),
                                 driver='GeoJSON',engine='pyogrio')

    # Also add the region as a final separate layer to union with, for wall-to-wall coverage
    # TODO: find a more generalized approach for this than topping off with a regional layer
    # of course could just omit the regional layer entirely and fillna on the parcel side
    buffered_stops['No_Fixed_Route_Transit'] = region_bdry

    # store the name as well
    source_frames_ordered.append('No_Fixed_Route_Transit')

    logger.info(
        '***Union the transit stop buffers to identify transit service areas***')

    #major_buf_name = 'Major_Transit_Stop'
    major_buf_name = source_frames_ordered[0]

    # From those stop buffers, build up the unions in a loop, starting with major stops
    prior_frame = buffered_stops[major_buf_name].rename(
        columns={'rowid': major_buf_name})

    i = 1
    for nme, current_frame in buffered_stops.items():

        if nme != major_buf_name:
            logger.info(
                f'\tUnion {i}: {"->".join(list(buffered_stops)[:i])}->{nme}')

            # in each loop we get a new prior_frame which is just the union
            # of past unions, and union with the current_frame
            prior_frame = gpd.overlay(prior_frame,
                                      current_frame.rename(
                                          columns={'rowid': nme}),
                                      how='union', keep_geom_type=True)
            # this is a topology hack to avoid linestring
            # TopologyException: found non-noded intersection between LINESTRING
            prior_frame['geometry'] = prior_frame['geometry'].buffer(0.01)
            i += 1

#     logger.info(
#         f'Result: a unioned frame of {len(prior_frame)} geographies')

    # in this frame, a missing value for a column means that a certain area is outside the
    # current column's transit buffer.
    prior_frame = (prior_frame
                   .fillna(-1)
                   .replace({0: True, -1: False}))

    # get a unique id for the combinations of presence / absence of different buffer types

    prior_frame['combination_id'] = prior_frame.groupby(
        source_frames_ordered).ngroup()

    # Then, for each combination of overlapping areas, get the most important
    # where rail (major stop) trumps shorter headways, which in turn trumps longer headways etc

    cat_count = len(source_frames_ordered)
    cat_count_var = f'cat{cat_count}'
    
    prior_frame[cat_count_var] = (prior_frame
                           .combination_id
                           .map(prior_frame
                                .groupby(['combination_id'])[source_frames_ordered]
                                .apply(
                                    get_combination_names_hierarchy))
                           )

    # Get the full string list of components in any given unioned area - might be in the vicinity
    # of a major stop, 15 min headway, 30 min headway, or just one or none of them. Which?

    prior_frame['combination_id_desc'] = (prior_frame
                                          .combination_id
                                          .map(prior_frame
                                               .groupby(['combination_id'])[source_frames_ordered]
                                               .apply(
                                                   get_combination_names))
                                          )

    # dissolve to just the core service category areas
    prior_frame_dissolved = prior_frame.dissolve([cat_count_var],
                                                 as_index=False)

    logger.info(
        f'Result: a unioned frame of {len(prior_frame)} component geographies, dissolved to {len(prior_frame_dissolved)}')

    if write_to_disk:
        logger.info(
            f'Writing gdf of transit service level areas to disk for {slug}')
        
        out_path = Path(today_path,f'transit_service_levels_{slug}.geojson')
        prior_frame_dissolved.to_file(
            out_path, driver='GeoJSON',engine='pyogrio')
        
        out_path = Path(today_path,f'transit_service_levels_{slug}.gpkg')
#         prior_frame_dissolved.to_file(
#             out_path, 
#             driver='GPKG',engine='pyogrio')


    # lastly, store a copy in this dict in the global namespace
    transit_buffers[slug] = prior_frame_dissolved
    return prior_frame, prior_frame_dissolved

## Step 1a: define filters

In [38]:
# set criteria for selecting headways and what kind of buffer to apply to selected stops

filter_criteria_major_stop = {'name': 'Major_Transit_Stop', 'buffer': .5,
                              'major_stop': [{'operator': '==', 'value': 1}]
                              }

filter_criteria_hdwy_lt15 = {'name': 'Bus_<15min', 'buffer': .25,
                             'am_av_hdwy': [{'operator': '<=', 'value': 15}],
                             'pm_av_hdwy': [{'operator': '<=', 'value': 15}]}


filter_criteria_hdwy_15_to_30 = {'name': 'Bus_15_30min', 'buffer': .25,
                                 'am_av_hdwy': [{'operator': '>', 'value': 15}, {'operator': '<=', 'value': 30}],
                                 'pm_av_hdwy': [{'operator': '>', 'value': 15}, {'operator': '<=', 'value': 30}]
                                 }

filter_criteria_hdwy_gt30 = {'name': 'Bus_30plusmin', 'buffer': .25,
                             'am_av_hdwy': [{'operator': '>', 'value': 30}],
                             'pm_av_hdwy': [{'operator': '>', 'value': 30}]}

criteria_list = [filter_criteria_major_stop,
                 filter_criteria_hdwy_lt15,
                 filter_criteria_hdwy_15_to_30,
                 filter_criteria_hdwy_gt30
                 ]

In [39]:
# set criteria for selecting headways and what kind of buffer to apply to selected stops
# ANUPs request

filter_criteria_major_stop = {'name': 'Major_Transit_Stop', 'buffer': .5,
                              'major_stop': [{'operator': '==', 'value': 1}]
                              }

filter_criteria_hdwy_lt10 = {'name': 'Bus_<10min', 'buffer': .25,
                             'am_av_hdwy': [{'operator': '<=', 'value': 10}],
                             'pm_av_hdwy': [{'operator': '<=', 'value': 10}]}


filter_criteria_hdwy_11_to_15 = {'name': 'Bus_11_15min', 'buffer': .25,
                                 'am_av_hdwy': [{'operator': '>', 'value': 10}, {'operator': '<=', 'value': 15}],
                                 'pm_av_hdwy': [{'operator': '>', 'value': 10}, {'operator': '<=', 'value': 15}]
                                 }
filter_criteria_hdwy_15_to_30 = {'name': 'Bus_15_30min', 'buffer': .25,
                                 'am_av_hdwy': [{'operator': '>', 'value': 15}, {'operator': '<=', 'value': 30}],
                                 'pm_av_hdwy': [{'operator': '>', 'value': 15}, {'operator': '<=', 'value': 30}]
                                 }

filter_criteria_hdwy_gt30 = {'name': 'Bus_30plusmin', 'buffer': .25,
                             'am_av_hdwy': [{'operator': '>', 'value': 30}],
                             'pm_av_hdwy': [{'operator': '>', 'value': 30}]}

criteria_list_anup = [filter_criteria_major_stop,
                      filter_criteria_hdwy_lt10,
                      filter_criteria_hdwy_11_to_15,
                      filter_criteria_hdwy_15_to_30,
                      filter_criteria_hdwy_gt30
                      ]

In [253]:
# set criteria for selecting headways and what kind of buffer to apply to selected stops
# ANUPs request


filter_criteria_hdwy_lt10 = {'name': 'FREQUENT TRANSIT', 'buffer': .5,
                             'am_av_hdwy': [{'operator': '<=', 'value': 10}],
                             'pm_av_hdwy': [{'operator': '<=', 'value': 10}]}



criteria_list_anup2 = [
                      filter_criteria_hdwy_lt10
                      ]

## Step 1: create transit buffers

### PBA50 5-way categorization

In [40]:
%%time 
transit_areas_current_cat5, transit_areas_current_dissolved_cat5 = create_multiple_transit_areas(
    transit_current,criteria_list, 'current_cat5',True)

transit_areas_fbp_cat5, transit_areas_fbp_dissolved_cat5 = create_multiple_transit_areas(
    transit_fbp,criteria_list, 'fbp_cat5',True)


transit_areas_np_cat5, transit_areas_np_dissolved_cat5 = create_multiple_transit_areas(
    transit_np,criteria_list, 'np_cat5',True)

03/22/2024 01:19:40 PM - INFO - ***Running create_transit_areas function for variant: current_cat5***
03/22/2024 01:19:40 PM - INFO - Transit Stop input geodataframe has shape: (40232, 18)
03/22/2024 01:19:40 PM - INFO - ***Looping through criteria list to subset transit stops***
03/22/2024 01:19:40 PM - INFO - Transit Station Subset: Major_Transit_Stop
03/22/2024 01:19:40 PM - INFO - 	Starting length of the dataframe: 40,232
03/22/2024 01:19:40 PM - INFO - 	Query string: (major_stop==1)
03/22/2024 01:19:40 PM - INFO - 	Records after subsetting: 4,802
03/22/2024 01:19:40 PM - INFO - 	Buffering a df of 4802 records with 0.50 miles
03/22/2024 01:19:43 PM - INFO - Transit Station Subset: Bus_<15min
03/22/2024 01:19:43 PM - INFO - 	Starting length of the dataframe: 40,232
03/22/2024 01:19:43 PM - INFO - 	Query string: (am_av_hdwy<=15) | (pm_av_hdwy<=15)
03/22/2024 01:19:43 PM - INFO - 	Records after subsetting: 15,832
03/22/2024 01:19:43 PM - INFO - 	Buffering a df of 15832 records with 0.

CPU times: user 44.9 s, sys: 1.22 s, total: 46.1 s
Wall time: 1min 55s


### PBA50Plus 6 way categorization

In [44]:
%%time 
transit_areas_current_cat6, transit_areas_current_dissolved_cat6 = create_multiple_transit_areas(
    transit_current,criteria_list_anup, 'current_cat6',True)

transit_areas_fbp_cat6, transit_areas_fbp_dissolved_cat6 = create_multiple_transit_areas(
    transit_fbp,criteria_list_anup, 'fbp_cat6',True)

transit_areas_np_cat6, transit_areas_np_dissolved_cat6 = create_multiple_transit_areas(
    transit_np,criteria_list_anup, 'np_cat6',True)

03/22/2024 01:25:52 PM - INFO - ***Running create_transit_areas function for variant: current_cat6***
03/22/2024 01:25:52 PM - INFO - Transit Stop input geodataframe has shape: (40232, 18)
03/22/2024 01:25:52 PM - INFO - ***Looping through criteria list to subset transit stops***
03/22/2024 01:25:52 PM - INFO - Transit Station Subset: Major_Transit_Stop
03/22/2024 01:25:52 PM - INFO - 	Starting length of the dataframe: 40,232
03/22/2024 01:25:52 PM - INFO - 	Query string: (major_stop==1)
03/22/2024 01:25:52 PM - INFO - 	Records after subsetting: 4,802
03/22/2024 01:25:52 PM - INFO - 	Buffering a df of 4802 records with 0.50 miles
03/22/2024 01:25:54 PM - INFO - Transit Station Subset: Bus_<10min
03/22/2024 01:25:54 PM - INFO - 	Starting length of the dataframe: 40,232
03/22/2024 01:25:54 PM - INFO - 	Query string: (am_av_hdwy<=10) | (pm_av_hdwy<=10)
03/22/2024 01:25:54 PM - INFO - 	Records after subsetting: 10,673
03/22/2024 01:25:54 PM - INFO - 	Buffering a df of 10673 records with 0.

CPU times: user 1min 13s, sys: 1.59 s, total: 1min 14s
Wall time: 2min 38s


### PBA50Plus 1 way categorization (major transit service)

In [261]:
%%time 
transit_areas_current_cat2, transit_areas_current_dissolved_cat2 = create_multiple_transit_areas(
    transit_current,criteria_list_anup2, 'current_cat2',True)

transit_areas_fbp_cat2, transit_areas_fbp_dissolved_cat2 = create_multiple_transit_areas(
    transit_fbp,criteria_list_anup2, 'fbp_cat2',True)


transit_areas_np_cat2, transit_areas_np_dissolved_cat2 = create_multiple_transit_areas(
    transit_np,criteria_list_anup2, 'np_cat2',True)

03/22/2024 04:49:55 PM - INFO - ***Running create_transit_areas function for variant: current_cat2***
03/22/2024 04:49:55 PM - INFO - Transit Stop input geodataframe has shape: (40232, 18)
03/22/2024 04:49:55 PM - INFO - ***Looping through criteria list to subset transit stops***
03/22/2024 04:49:55 PM - INFO - Transit Station Subset: FREQUENT TRANSIT
03/22/2024 04:49:55 PM - INFO - 	Starting length of the dataframe: 40,232
03/22/2024 04:49:55 PM - INFO - 	Query string: (am_av_hdwy<=10) | (pm_av_hdwy<=10)
03/22/2024 04:49:55 PM - INFO - 	Records after subsetting: 10,673
03/22/2024 04:49:55 PM - INFO - 	Buffering a df of 10673 records with 0.50 miles
03/22/2024 04:49:58 PM - INFO - ***Union the transit stop buffers to identify transit service areas***
03/22/2024 04:49:58 PM - INFO - 	Union 1: FREQUENT TRANSIT->No_Fixed_Route_Transit
03/22/2024 04:49:59 PM - INFO - Result: a unioned frame of 2 component geographies, dissolved to 2
03/22/2024 04:49:59 PM - INFO - Writing gdf of transit se

CPU times: user 8.98 s, sys: 124 ms, total: 9.11 s
Wall time: 22.7 s


### Inspect

In [49]:
# mine_vs_bobbys_fbp = gpd.overlay(v2020_fbp_classes.to_crs(ANALYSIS_CRS),transit_areas_fbp_dissolved_cat5)

# mine_vs_bobbys_fbp['area'] = mine_vs_bobbys_fbp.area

# def pct(x): return x/x.sum()

# mine_vs_bobbys = mine_vs_bobbys_fbp.groupby(['Service_Level', 'cat5']).area.sum(
# ).groupby(level=0, observed=True).apply(pct).round(2).unstack()

# # While the diagonal is clearly there, there are important differences. Why?
# # Turns out there are data source differences between the 2000 version and the one on AGOL now. Unclear why.
# sns.heatmap(mine_vs_bobbys, cmap='coolwarm', annot=True)

In [50]:
# missing a gdal driver on my box
# transit_areas_np_dissolved_cat6.to_file(new_fgdb, layer="transit_areas_np_dissolved_cat6", driver="FileGDB")

## Step 2 - relate transit proximities to parcels (before adding any run specific data)

In [256]:
def assign_transit_service_areas_to_parcels(parcels_geo, transit_areas, slug='current', variable='cat5'):

    logger.info(f'Joining transit areas ({slug}) to parcels')

    # core assignment, per sjoin
    p10_x_transit_areas = gpd.sjoin(
        parcels_geo, transit_areas, predicate='within')

    # check for unassigned
    unassigned_parcel_ids = set(
        parcels_geo.PARCEL_ID)-set(p10_x_transit_areas.PARCEL_ID)

    logger.info(
        f'There were {len(unassigned_parcel_ids)} parcels without an assignment to a transit area.\nWe assign to nearest area.')

    p10_x_transit_areas_remainder = gpd.sjoin_nearest(parcels_geo.query(
        'PARCEL_ID.isin(@unassigned_parcel_ids)'), transit_areas)

    # combine the two
    combo_assignments = pd.concat(
        [p10_x_transit_areas, p10_x_transit_areas_remainder])

    if (combo_assignments.PARCEL_ID.value_counts().sort_values() > 1).any():

        # if there are parcel duplicates - get the fist classification
        mapping = combo_assignments.groupby(
            'PARCEL_ID')[variable].first()
    else:
        # return the original mapping

        mapping = combo_assignments.set_index('PARCEL_ID')[variable]

    return mapping

### Assign service areas
#### cat5

In [52]:
%%time

p10_x_transit_area_np_cat5 = assign_transit_service_areas_to_parcels(parcels_geo=p10_topofix_pt, transit_areas=transit_areas_np_dissolved_cat5,slug= 'np', variable='cat5')
p10_x_transit_area_fbp_cat5 = assign_transit_service_areas_to_parcels(p10_topofix_pt, transit_areas=transit_areas_fbp_dissolved_cat5, slug='fbp', variable='cat5')
p10_x_transit_area_current_cat5 = assign_transit_service_areas_to_parcels(p10_topofix_pt, transit_areas=transit_areas_current_dissolved_cat5, slug='current', variable='cat5')

03/22/2024 01:36:01 PM - INFO - Joining transit areas (np) to parcels
03/22/2024 01:36:11 PM - INFO - There were 72 parcels without an assignment to a transit area.
We assign to nearest area.
03/22/2024 01:36:13 PM - INFO - Joining transit areas (fbp) to parcels
03/22/2024 01:36:21 PM - INFO - There were 72 parcels without an assignment to a transit area.
We assign to nearest area.
03/22/2024 01:36:24 PM - INFO - Joining transit areas (current) to parcels
03/22/2024 01:36:32 PM - INFO - There were 72 parcels without an assignment to a transit area.
We assign to nearest area.


CPU times: user 30.9 s, sys: 2.02 s, total: 33 s
Wall time: 33.1 s


#### cat6

In [55]:
%%time
p10_x_transit_area_np_cat6 = assign_transit_service_areas_to_parcels(parcels_geo=p10_topofix_pt, transit_areas=transit_areas_np_dissolved_cat6, slug='np', variable='cat6')
p10_x_transit_area_fbp_cat6 = assign_transit_service_areas_to_parcels(parcels_geo=p10_topofix_pt, transit_areas=transit_areas_fbp_dissolved_cat6,slug= 'fbp', variable='cat6')
p10_x_transit_area_current_cat6 = assign_transit_service_areas_to_parcels(parcels_geo=p10_topofix_pt, transit_areas=transit_areas_current_dissolved_cat6, slug='current', variable='cat6')

03/22/2024 01:37:38 PM - INFO - Joining transit areas (np) to parcels
03/22/2024 01:37:48 PM - INFO - There were 72 parcels without an assignment to a transit area.
We assign to nearest area.
03/22/2024 01:37:50 PM - INFO - Joining transit areas (fbp) to parcels
03/22/2024 01:37:59 PM - INFO - There were 72 parcels without an assignment to a transit area.
We assign to nearest area.
03/22/2024 01:38:02 PM - INFO - Joining transit areas (current) to parcels
03/22/2024 01:38:11 PM - INFO - There were 72 parcels without an assignment to a transit area.
We assign to nearest area.


CPU times: user 33.7 s, sys: 1.9 s, total: 35.6 s
Wall time: 35.7 s


#### cat2 frequent transit

In [262]:
%%time
p10_x_transit_area_np_cat2 = assign_transit_service_areas_to_parcels(parcels_geo=p10_topofix_pt, transit_areas=transit_areas_np_dissolved_cat1, slug='np', variable='cat2')
p10_x_transit_area_fbp_cat2 = assign_transit_service_areas_to_parcels(parcels_geo=p10_topofix_pt, transit_areas=transit_areas_fbp_dissolved_cat1,slug= 'fbp', variable='cat2')
p10_x_transit_area_current_cat2 = assign_transit_service_areas_to_parcels(parcels_geo=p10_topofix_pt, transit_areas=transit_areas_current_dissolved_cat1, slug='current', variable='cat2')

03/22/2024 04:50:19 PM - INFO - Joining transit areas (np) to parcels
03/22/2024 04:50:28 PM - INFO - There were 72 parcels without an assignment to a transit area.
We assign to nearest area.
03/22/2024 04:50:30 PM - INFO - Joining transit areas (fbp) to parcels
03/22/2024 04:50:39 PM - INFO - There were 72 parcels without an assignment to a transit area.
We assign to nearest area.
03/22/2024 04:50:41 PM - INFO - Joining transit areas (current) to parcels
03/22/2024 04:50:50 PM - INFO - There were 72 parcels without an assignment to a transit area.
We assign to nearest area.


CPU times: user 29.5 s, sys: 2.81 s, total: 32.3 s
Wall time: 32.4 s


### Collect in parcel - x - service areas in dict

In [265]:
# store the parcels-to-transit service level correspondences in a dict for easy retrieval

transit_scenario_crosswalk = {'cur_cat5': p10_x_transit_area_current_cat5,
                              'fbp_cat5': p10_x_transit_area_fbp_cat5,
                              'np_cat5': p10_x_transit_area_np_cat5,
                              'cur_cat6': p10_x_transit_area_current_cat6,
                              'fbp_cat6': p10_x_transit_area_fbp_cat6,
                              'np_cat6': p10_x_transit_area_np_cat6,
                              'cur_cat2': p10_x_transit_area_current_cat2,
                              'fbp_cat2': p10_x_transit_area_fbp_cat2,
                              'np_cat2': p10_x_transit_area_np_cat2,
                             }

## Step 3 Classify topo parcels to transit service areas and tracts and epc based on parcel_id


In [266]:
def add_classifications_to_p10_topo(p10_topofix_pt):

    ####################################################
    # add tractids, tazs to topo parcels

    # vintage 2010
    p10_topofix_pt['tract10'] = p10_topofix_pt.PARCEL_ID.map(
        parcel_x_tracts.set_index('parcel_id').tract10.map(lambda x: f'{x:011.0f}'))
    col_values(p10_topofix_pt['tract10'], 'tract10')

    # vintage 2020
    p10_topofix_pt['tract20'] = p10_topofix_pt.PARCEL_ID.map(
        parcel_x_tracts.set_index('parcel_id').tract20.map(lambda x: f'{x:011.0f}'))
    col_values(p10_topofix_pt['tract20'], 'tract20')

    # add TAZs
    p10_topofix_pt['taz'] = p10_topofix_pt.PARCEL_ID.map(
        parcel_x_taz.set_index('PARCEL_ID').ZONE_ID)
    col_values(p10_topofix_pt['taz'], 'taz')

    ####################################################

    # add area type based on the just assigned taz
    p10_topofix_pt['area_type'] = p10_topofix_pt.taz.map(taz_x_areatype)
    col_values(p10_topofix_pt['area_type'], 'area_type')

    # add epc (RTP2021 version) to parcels using tract10
    p10_topofix_pt['epc21'] = p10_topofix_pt.tract10.map(
        tract_epc21_file.set_index('geoid10').epc_class.fillna('Not EPC'))
    col_values(p10_topofix_pt['epc21'], 'epc21')

    p10_topofix_pt['is_epc21'] = p10_topofix_pt.epc21 != 'Not EPC'
    col_values(p10_topofix_pt['is_epc21'], 'is_epc21')

    # add epc (RTP2025 version) to parcels using tract10
    p10_topofix_pt['epc24'] = p10_topofix_pt.tract20.map(
        tract_epc24_file.set_index('geoid20').epc_class.fillna('Not EPC'))
    col_values(p10_topofix_pt['epc24'], 'epc24')

    p10_topofix_pt['is_epc24'] = p10_topofix_pt.epc24 != 'Not EPC'
    col_values(p10_topofix_pt['is_epc24'], 'is_epc24')

    # add hra23 to parcels using tract10 (!)
    p10_topofix_pt['hra23'] = p10_topofix_pt.tract10.map(
        tract_hra_2023.set_index('geoid').hra_category)
    col_values(p10_topofix_pt['hra23'], 'hra23')

    p10_topofix_pt['is_hra23'] = p10_topofix_pt.hra23.isin(
        ["High Resource", "Highest Resource"])
    col_values(p10_topofix_pt['is_hra23'], 'is_hra23')

    # add hra19 to parcels using tract10
    p10_topofix_pt['hra19'] = p10_topofix_pt.tract10.map(
        tract_hra_2019.set_index('geoid').hra_category)
    col_values(p10_topofix_pt['hra19'], 'hra19')

    p10_topofix_pt['is_hra19'] = p10_topofix_pt.hra19.isin(
        ["High Resource", "Highest Resource"])
    col_values(p10_topofix_pt['is_hra19'], 'is_hra19')

#     # map the cat5 service level names to more descriptive ones used in Tableau
#     p10_topofix_pt['Service_Level'] = p10_topofix_pt.PARCEL_ID.map(
#         transit_scenario_crosswalk[transit_scenario])
#     col_values(p10_topofix_pt['Service_Level'], 'Service_Level')

    for key, val in transit_scenario_crosswalk.items():
        service_level_var = f'Service_Level_{key}'

        p10_topofix_pt[service_level_var] = p10_topofix_pt.PARCEL_ID.map(
            transit_scenario_crosswalk[key])
        col_values(p10_topofix_pt[service_level_var], service_level_var)

    return p10_topofix_pt


p10_topofix_pt2 = add_classifications_to_p10_topo(p10_topofix_pt)

03/22/2024 04:51:04 PM - INFO - tract10 records with no value assigned: 1
03/22/2024 04:51:07 PM - INFO - tract20 records with no value assigned: 1
03/22/2024 04:51:07 PM - INFO - taz records with no value assigned: 0
03/22/2024 04:51:07 PM - INFO - area_type records with no value assigned: 0
03/22/2024 04:51:08 PM - INFO - epc21 records with no value assigned: 18
03/22/2024 04:51:08 PM - INFO - is_epc21 records with no value assigned: 0
03/22/2024 04:51:09 PM - INFO - epc24 records with no value assigned: 14
03/22/2024 04:51:09 PM - INFO - is_epc24 records with no value assigned: 0
03/22/2024 04:51:10 PM - INFO - hra23 records with no value assigned: 153139
03/22/2024 04:51:10 PM - INFO - is_hra23 records with no value assigned: 0
03/22/2024 04:51:10 PM - INFO - hra19 records with no value assigned: 153139
03/22/2024 04:51:11 PM - INFO - is_hra19 records with no value assigned: 0
03/22/2024 04:51:11 PM - INFO - Service_Level_cur_cat5 records with no value assigned: 1
03/22/2024 04:51:

In [210]:
p10_topofix_pt2[p10_topofix_pt2.hra19.notna()].to_file('/Users/aolsen/Downloads/p10_topofix_pt2_Hra19missing.geojson',driver='GeoJSON')

In [211]:
# # only 3 tracts in parcels are not in the tract file
# #parcel to tract crosswalking is impeccable 
# set(p10_topofix_pt2.tract10)-set(tract_2010.GEOID10)

# set(tract_2010.GEOID10) - set(p10_topofix_pt2.tract10)

# set(p10_topofix_pt2.tract20)-set(tract_2020.GEOID)

# set(tract_2020.GEOID) - set(p10_topofix_pt2.tract20)

# # show tracts in parcels where hra19 couldn't be assigned
# set(p10_topofix_pt2[p10_topofix_pt2.hra19.isna()].tract10) - \
#     set(tract_hra_2019.geoid)

In [212]:
# set(p10_topofix_pt2[p10_topofix_pt2.hra19.isna()].tract10) - \
#     set(tract_2010.GEOID10)

In [213]:
# tract_2010.query('GEOID10=="06097150303"').explore()

In [214]:
# tract_hra_2019.set_index('geoid').hra_category.loc['06081613501']

In [216]:
# p10_topofix_pt.tract10  # .map(tract_hra_2023.set_index('geoid').hra_category)

In [66]:
%%time
p10_topofix_pt2.to_feather(Path(WORKING_DIR,'outputs',YDM,'p10_topofix_pt2.feather'))

CPU times: user 3.74 s, sys: 621 ms, total: 4.36 s
Wall time: 1min 20s


In [67]:
%%time
p10_topofix_pt2.to_parquet(Path(WORKING_DIR,'outputs',YDM,'p10_topofix_pt2.parquet'))

CPU times: user 4.55 s, sys: 355 ms, total: 4.91 s
Wall time: 36.3 s


In [76]:
%%time
p10_topofix_pt3 = gpd.read_parquet(Path(WORKING_DIR,'outputs',YDM,'p10_topofix_pt2.parquet'))

CPU times: user 2.77 s, sys: 632 ms, total: 3.41 s
Wall time: 10.3 s


## Step 4: run the summaries
Note that we want to do the proper matching:
* 2015 should always be "current"
* 2050 blueprint should be matched with 2050 blueprint transit buffers
* 2050 no project should be matched with 2050 no project transit buffers

We are looking for output as follows [metrics_proximity.csv](https://mtcdrive.app.box.com/file/837349294434)


In [217]:
metrics_proximity_target_schema = pd.read_csv(
    '/Users/aolsen/Box/Horizon and Plan Bay Area 2050/Equity and Performance/7_Analysis/Metrics/Metrics_Outputs_FinalBlueprint/Intermediate Metrics/Archive Draft Plan Mar 2021/metrics_proximity.csv')
schema_iteration_vars = ['Service_Level', 'year',
                         'modelrunID', 'transit', 'area', 'blueprint']
# metrics_proximity_target_schema.groupby(schema_iteration_vars).size().unstack('modelrunID')

In [218]:
def col_values(s, colname):
    #msg = f'values for {colname}\n n{s.value_counts()}'
    msg2 = f'{colname} records with no value assigned: {len(s[s.isna()])}'
    # logger.info(msg)
    logger.info(msg2)


col_values(parcel_x_tracts.tract10, 'trt')

03/22/2024 03:31:58 PM - INFO - trt records with no value assigned: 17


In [231]:
run_mapping_parcels = {
    'PBA50Plus_DBP_InitialRun': {'run_path':  'PBA50Plus_DBP_InitialRun_v6',
                                 'mnemonic': 'PBA50Plus_DBP_InitialRun_v6',
                                 'root_path': '/Volumes/Data/Models/urban_modeling/baus/PBA50Plus/PBA50Plus_DBP_InitialRun/outputs'},

    'PBA50Plus_NP_InitialRun': {'run_path':  'PBA50Plus_NP_InitialRun_v7',
                                'mnemonic': 'PBA50Plus_NP_InitialRun_v7',
                                'root_path': '/Volumes/Data/Models/urban_modeling/baus/PBA50Plus/PBA50Plus_NP_InitialRun/outputs'},
    'PBA50_NP': {'run_path':  'EIR runs/Baseline Large (s25) runs/NP_v8_FINAL',
                 'mnemonic': 'PBA50_NP',
                 'run_name_stub': 'run314',
                 'root_path': urbansim_run_location
                 },
    'PBA50_FBP': {'run_path':  'Final Blueprint runs/Final Blueprint (s24)/BAUS v2.25 - FINAL VERSION',
                  'mnemonic': 'PBA50_FBP',
                  'run_name_stub': 'run182',
                  'root_path': urbansim_run_location
                  },
}

In [232]:
parcel_cols_keep = ['parcel_id', 'tothh',
                    'hhq1', 'totemp', 'RETEMPN', 'MWTEMPN']


def parcel_file_name(run_mnemonic, year):

    if 'run_name_stub' in run_mapping_parcels[run_mnemonic]:
        print(1)
        parcel_template = '{}_parcel_data_{}.csv'.format(
            run_mapping_parcels[run_mnemonic]['run_name_stub'], year)
    else:
        parcel_template = 'parcel_data_{}.csv'.format(year)
    out_path = Path(run_mapping_parcels[run_mnemonic]['root_path'],
                    run_mapping_parcels[run_mnemonic]['run_path'], parcel_template)
    return out_path


parcel_output = pd.read_csv(parcel_file_name(
    'PBA50_FBP', 2050), usecols=parcel_cols_keep)

1


In [267]:
# p10_topofix_pt_mrg = p10_topofix_pt2.merge(
#     parcel_output, left_on=['PARCEL_ID'], right_on=['parcel_id'])

In [268]:
# p10_topofix_pt_mrg.filter(regex='cat6')

In [274]:
def summarize_parcels(parcel_output, year, transit_scenario):
    """Summarizes parcel data by transit service levels and geographic groupings.

    This function reads a DataFrame containing parcel information, prepares it for analysis,
    and then calculates summaries based on transit service levels, Equity Priority
    Communities (EPCs), High Resource Areas (HRAs), and area types.

    Args:
      parcel_output: A pandas DataFrame containing information about parcels.
      year: The year for which the summaries are generated (likely used for filtering data).
      transit_scenario: The transit scenario to focus on (e.g., 'fbp' for frequent bus service).

    Returns:
      A pandas DataFrame with summaries for various transit service levels, EPCs, HRAs, and area types.
    """

    # Read or assume parcel data is already loaded in parcel_output
    # logger.info(' Read {} rows'.format(len(parcel_output)))

    # Drop extraneous cols
    # parcel_output.drop(['geom_id', 'total_job_spaces', 'zoned_du',
    #                    'zoned_du_underbuild', 'zoned_du_underbuild_nodev', 'first_building_type'], axis=1, inplace=True)

    # Log informative message about data
    # logger.info("Head:\n{}".format(parcel_output.head()))

    # Define columns containing values of interest
    val_cols = ['totemp', 'RETEMPN', 'MWTEMPN', 'tothh']

    # Fill missing values and convert specific columns to integers for consistency
    for col in val_cols:
        parcel_output[col] = parcel_output[col].fillna(0).round(0).astype(int)

    # Merge parcel run data geographic classifiers / categorizations
    p10_topofix_pt_mrg = p10_topofix_pt2.merge(
        parcel_output, left_on=['PARCEL_ID'], right_on=['parcel_id'])

    #return p10_topofix_pt_mrg
    def groupby_summaries(df, group_vars):

        grp_summary = (df
                       .groupby(group_vars)
                       .agg({'tothh': 'sum', 'hhq1': 'sum',
                             'totemp': 'sum', 'RETEMPN': 'sum',
                             'MWTEMPN': 'sum'})
                       )

        grp_summary_shares = grp_summary.div(
            grp_summary.sum()).round(3)

        grp_summary_combo = pd.concat([grp_summary,
                                       grp_summary_shares.add_suffix('_share')
                                       ], axis=1)
        return grp_summary_combo

    # Identify the passed scenario-specific columns (fbp,no project, current)
    # this returns different classifications for each - like the 5-way or 6-way service level (cat5, cat6) 
    # we summarize run data for each classification variable
    transit_svcs_cols = p10_topofix_pt_mrg.filter(
        regex=transit_scenario).columns

    container = {}

    # Calculate summaries for various groupings
    for var in transit_svcs_cols:
        summary_servicelevel = groupby_summaries(p10_topofix_pt_mrg, var)

        summary_servicelevel_epc = groupby_summaries(
            p10_topofix_pt_mrg, ['is_epc24', var])

        summary_servicelevel_hra = groupby_summaries(
            p10_topofix_pt_mrg, ['is_hra23', var])

        summary_servicelevel_areatype = groupby_summaries(
            p10_topofix_pt_mrg, ['area_type', var])

        container[('Region',var)] = pd.concat([summary_servicelevel], keys=['reg'])
        container[('CoCs',var)] = summary_servicelevel_epc
        container[('HRAs',var)] = summary_servicelevel_hra
        container[('area_type',var)] = summary_servicelevel_areatype
    return pd.concat(container)


parcel_output_summary_trn_fp_cat5 = summarize_parcels(
    parcel_output, 2050, 'fbp')

In [275]:
parcel_output_summary_trn_fp_cat5.loc['Region']

tothh      hhq1  \
Service_Level_fbp_cat5 reg Bus_15_30min_only         474217   66392.0   
                           Bus_30plusmin_only        229297   27901.0   
                           Bus_<15min_only           565775  135653.0   
                           Major_Transit_Stop_only  1970295  818833.0   
                           none                      801973   93401.0   
Service_Level_fbp_cat6 reg Bus_11_15min_only         173820   42198.0   
                           Bus_15_30min_only         474217   66392.0   
                           Bus_30plusmin_only        229297   27901.0   
                           Bus_<10min_only           391960   93457.0   
                           Major_Transit_Stop_only  1970291  818832.0   
                           none                      801972   93400.0   
Service_Level_fbp_cat2 reg FREQUENT TRANSIT_only    2458809  913657.0   
                           none                     1582748  228523.0   

                                                     totemp  RETEMPN  MWTEMPN  \
Service_Level_fbp_cat5 reg Bus_15_30min_only         838520    78434   123208   
                           Bus_30plusmin_only        316267    26679    41895   
                           Bus_<15min_only           674561    75155    73564   
                           Major_Transit_Stop_only  2780290   221425   192785   
                           none                      797175    39862   176197   
Service_Level_fbp_cat6 reg Bus_11_15min_only         226703    18103    24937   
                           Bus_15_30min_only         838393    78398   123206   
                           Bus_30plusmin_only        316267    26679    41895   
                           Bus_<10min_only           447858    57052    48627   
                           Major_Transit_Stop_only  2780417   221461   192787   
                           none                      797175    39862   176197   
Service_Level_fbp_cat2 reg FREQUENT TRANSIT_only    3329219   288934   271425   
                           none                     2077594   152621   336224   

                                                    tothh_share  hhq1_share  \
Service_Level_fbp_cat5 reg Bus_15_30min_only              0.117       0.058   
                           Bus_30plusmin_only             0.057       0.024   
                           Bus_<15min_only                0.140       0.119   
                           Major_Transit_Stop_only        0.488       0.717   
                           none                           0.198       0.082   
Service_Level_fbp_cat6 reg Bus_11_15min_only              0.043       0.037   
                           Bus_15_30min_only              0.117       0.058   
                           Bus_30plusmin_only             0.057       0.024   
                           Bus_<10min_only                0.097       0.082   
                           Major_Transit_Stop_only        0.488       0.717   
                           none                           0.198       0.082   
Service_Level_fbp_cat2 reg FREQUENT TRANSIT_only          0.608       0.800   
                           none                           0.392       0.200   

                                                    totemp_share  \
Service_Level_fbp_cat5 reg Bus_15_30min_only               0.155   
                           Bus_30plusmin_only              0.058   
                           Bus_<15min_only                 0.125   
                           Major_Transit_Stop_only         0.514   
                           none                            0.147   
Service_Level_fbp_cat6 reg Bus_11_15min_only               0.042   
                           Bus_15_30min_only               0.155   
                           Bus_30plusmin_only              0.058   
                           Bus_<10min_only                 0.083   
                           Major_Transit_Stop_only         0.514   
                           none          

In [ ]:
parcel_output_summary_trn_fp_cat6 = summarize_parcels(
    parcel_output, 2050, 'fbp_cat6')
parcel_output_summary_trn_np_cat5 = summarize_parcels(
    parcel_output, 2050, 'trn_np_cat5')
parcel_output_summary_trn_np_cat6 = summarize_parcels(
    parcel_output, 2050, 'trn_np_cat6')
parcel_output_summary_trn_cur_cat5 = summarize_parcels(
    parcel_output, 2050, 'trn_cur_cat5')
parcel_output_summary_trn_cur_cat6 = summarize_parcels(
    parcel_output, 2050, 'trn_cur_cat6')

In [60]:
parcel_output_summary

NameError: name 'parcel_output_summary' is not defined

In [602]:
# copying in the code from the metrics script for reference - 
# it is mainly an accounting script in the sense that the outputs are really coming from Bobby's scripts
# and by extension this notebook's scripts - share of households, etc by different transit service geographes

def calculate_Connected1_proximity(runid, year, dbp, transitproximity_df, metrics_dict):

    metric_id = "C1"

    for area_type in ['Region', 'CoCs', 'HRAs', 'rural', 'suburban', 'urban']:
        # households
        metrics_dict[runid, metric_id, 'transitproximity_majorstop_shareof_tothh_%s' % area_type, year, dbp] = transitproximity_df.loc[(transitproximity_df['Service_Level'] == "Major_Transit_Stop")
                                                                                                                                       & (transitproximity_df['year'] == int(year))
                                                                                                                                       & (transitproximity_df['blueprint'].str.contains(dbp))
                                                                                                                                       & (transitproximity_df['area'] == area_type), 'tothh_share'].sum()
        metrics_dict[runid, metric_id, 'transitproximity_majorstop_shareof_hhq1_%s' % area_type, year, dbp] = transitproximity_df.loc[(transitproximity_df['Service_Level'] == "Major_Transit_Stop")
                                                                                                                                      & (transitproximity_df['year'] == int(year))
                                                                                                                                      & (transitproximity_df['blueprint'].str.contains(dbp))
                                                                                                                                      & (transitproximity_df['area'] == area_type), 'hhq1_share'].sum()
        # jobs
        metrics_dict[runid, metric_id, 'transitproximity_majorstop_shareof_totemp_%s' % area_type, year, dbp] = transitproximity_df.loc[(transitproximity_df['Service_Level'] == "Major_Transit_Stop")
                                                                                                                                        & (transitproximity_df['year'] == int(year))
                                                                                                                                        & (transitproximity_df['blueprint'].str.contains(dbp))
                                                                                                                                        & (transitproximity_df['area'] == area_type), 'totemp_share'].sum()
        metrics_dict[runid, metric_id, 'transitproximity_majorstop_shareof_RETEMPNjobs_%s' % area_type, year, dbp] = transitproximity_df.loc[(transitproximity_df['Service_Level'] == "Major_Transit_Stop")
                                                                                                                                             & (transitproximity_df['year'] == int(year))
                                                                                                                                             & (transitproximity_df['blueprint'].str.contains(dbp))
                                                                                                                                             & (transitproximity_df['area'] == area_type), 'RETEMPN_share'].sum()
        metrics_dict[runid, metric_id, 'transitproximity_majorstop_shareof_MWTEMPNjobs_%s' % area_type, year, dbp] = transitproximity_df.loc[(transitproximity_df['Service_Level'] == "Major_Transit_Stop")
                                                                                                                                             & (transitproximity_df['year'] == int(year)) & (transitproximity_df['blueprint'].str.contains(dbp))
                                                                                                                                             & (transitproximity_df['area'] == area_type), 'MWTEMPN_share'].sum()

In [266]:
metrics = pd.read_csv('/Users/aolsen/Box/Horizon and Plan Bay Area 2050/Equity and Performance/7_Analysis/Metrics/Metrics_Outputs_FinalBlueprint/metrics.csv')
metrics.loc[(metrics.modelrunID.str.contains("PlusCrossing"))&(metrics['name'].str.contains('transitprox'))]

,modelrunID,metric,name,year,blueprint,value
2640,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_tothh_Region,2050,Plus,0.49
2641,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_hhq1_Region,2050,Plus,0.74
2642,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_totemp_Region,2050,Plus,0.51
2643,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_RETEMPNjobs...,2050,Plus,0.5
2644,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_MWTEMPNjobs...,2050,Plus,0.32
2645,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_tothh_CoCs,2050,Plus,0.68
2646,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_hhq1_CoCs,2050,Plus,0.82
2647,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_totemp_CoCs,2050,Plus,0.66
2648,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_RETEMPNjobs...,2050,Plus,0.61
2649,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_MWTEMPNjobs...,2050,Plus,0.4


In [263]:
metrics.query('blueprint=="NoProject" & name=="deed_restricted_total"')

,modelrunID,metric,name,year,blueprint,value
168,EIR runs/Baseline Large (s25) runs/NP_v8_FINAL...,A2,deed_restricted_total,2050,NoProject,574252
169,EIR runs/Baseline Large (s25) runs/NP_v8_FINAL...,A2,deed_restricted_total,2015,NoProject,121292


In [264]:
# metrics[metrics.name.str.lower().str.contains('deed')].set_index(['modelrunID','year','blueprint','name']).value.unstack([0,2]).loc[2050].T.to_clipboard()

In [267]:
metrics_proximity_target_schema.query('blueprint=="Plus" & year==2050 & transit=="trn_fp_cat5" & area=="Region"') #& area=="CoCs"

,Service_Level,tothh,hhq1,totemp,RETEMPN,MWTEMPN,tothh_share,hhq1_share,totemp_share,RETEMPN_share,MWTEMPN_share,year,modelrunID,transit,area,blueprint
165,Bus_15_30min,241445,30886,415937,36569,68977,0.06,0.03,0.08,0.08,0.11,2050,Final Blueprint runs/Final Blueprint (s24)/BAU...,trn_fp_cat5,Region,Plus
166,Bus_31plus_min,281742,31131,379888,31432,60476,0.07,0.03,0.07,0.07,0.10,2050,Final Blueprint runs/Final Blueprint (s24)/BAU...,trn_fp_cat5,Region,Plus
167,Bus_<15min,786845,127837,1083359,115284,114261,0.19,0.13,0.20,0.26,0.19,2050,Final Blueprint runs/Final Blueprint (s24)/BAU...,trn_fp_cat5,Region,Plus
168,Major_Transit_Stop,1971306,749222,2785232,221557,194184,0.49,0.74,0.51,0.50,0.32,2050,Final Blueprint runs/Final Blueprint (s24)/BAU...,trn_fp_cat5,Region,Plus
169,No_Fixed_Route_Transit,761974,70889,744044,36717,169760,0.19,0.07,0.14,0.08,0.28,2050,Final Blueprint runs/Final Blueprint (s24)/BAU...,trn_fp_cat5,Region,Plus


In [280]:
urbansim_run_location = f'/Users/{user}/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/'
# keep DBP for future
us_2050_DBP_Final = 'Draft Blueprint runs/Blueprint Plus Crossing (s23)/v1.7.1- FINAL DRAFT BLUEPRINT/run98'
us_2050_FBP_Final = 'Final Blueprint runs/Final Blueprint (s24)/BAUS v2.25 - FINAL VERSION/run182'
us_2050_NP_EIR = 'EIR runs/Baseline Large (s25) runs/NP_v8_FINAL/run314'
us_2050_ALT1_EIR = 'EIR runs/Alt1 (s26) runs/Alt1_v3_test_far_tiers_FINAL_EIR_ALT/run375'
us_2050_ALT2_EIR = 'EIR runs/Alt2 (s28) runs/Alt2_v1_FINAL_EIR_ALT/run374'

list_us_runid = [us_2050_NP_EIR, us_2050_ALT1_EIR, us_2050_ALT2_EIR,
                 us_2050_FBP_Final]  # , us_2050_FBP_Final, us_2050_DBP_Final]
list_us_runid

['EIR runs/Baseline Large (s25) runs/NP_v8_FINAL/run314',
 'EIR runs/Alt1 (s26) runs/Alt1_v3_test_far_tiers_FINAL_EIR_ALT/run375',
 'EIR runs/Alt2 (s28) runs/Alt2_v1_FINAL_EIR_ALT/run374',
 'Final Blueprint runs/Final Blueprint (s24)/BAUS v2.25 - FINAL VERSION/run182']

In [321]:
run_mapping_parcels['PBA50_FBP']['root_path']

'/Users/aolsen/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/'

In [310]:
model_year=2050
us_2050_FBP_Final = 'EIR runs/Baseline Large (s25) runs/NP_v8_FINAL'
run_id = 'run314'

parcels_np50 = pd.read_csv(Path(urbansim_run_location,us_2050_FBP_Final,'{}_parcel_data_{}.csv'.format(run_id,model_year)))

In [304]:
parcels_pba50.deed_restricted_units.sum()/parcels_pba50.residential_units.sum()
parcels_pba50.residential_units.sum()

4259618.0

In [275]:

us_2050_FBP_Final

'Final Blueprint runs/Final Blueprint (s24)/BAUS v2.25 - FINAL VERSION/run182'

In [ ]:
model_year=2050
us_2050_FBP_Final = 'Final Blueprint runs/Final Blueprint (s24)/BAUS v2.25 - FINAL VERSION'
run_id = 'run182'

parcels_pba50 = pd.read_csv(Path(urbansim_run_location,us_2050_FBP_Final,'{}_parcel_data_{}_UBI.csv'.format(run_id,model_year)))

In [ ]:
def load_pba_parcel_data(scenario):

    column_map = {'tothh_share', 'hhq1_share',
                  'totemp_share', 'RETEMPN_share', 'MWTEMPN_share'}

    for model_year in (2015, 2050):
        if model_year == 2050:
            if us_runid == us_2050_FBP_Final:
                parcel_file = urbansim_runid + \
                    '_parcel_data_{}_UBI.csv'.format(model_year)
            else:
                parcel_file = urbansim_runid + \
                    '_parcel_data_{}.csv'.format(model_year)
        else:
            parcel_file = urbansim_runid + \
                '_parcel_data_{}.csv'.format(model_year)
        logger.info('Reading {} parcel data from {}'.format(
            model_year, parcel_file))
        parcel_output = pd.read_csv(parcel_file, engine='python')

        logger.info('  Read {} rows'.format(len(parcel_output)))

        # keep essential columns
        parcel_output.drop(['geom_id', 'total_job_spaces', 'zoned_du',
                            'zoned_du_underbuild', 'zoned_du_underbuild_nodev', 'first_building_type'], axis=1, inplace=True)
        logger.info("Head:\n{}".format(parcel_output.head()))

        parcel_output['totemp'] = parcel_output['totemp'].fillna(0)
        parcel_output['totemp'] = parcel_output['totemp'].round(
            0).astype('int')
        parcel_output['RETEMPN'] = parcel_output['RETEMPN'].fillna(0)
        parcel_output['RETEMPN'] = parcel_output['RETEMPN'].round(
            0).astype('int')
        parcel_output['MWTEMPN'] = parcel_output['MWTEMPN'].fillna(0)
        parcel_output['MWTEMPN'] = parcel_output['MWTEMPN'].round(
            0).astype('int')

['tothh', 'hhq1', 'totemp', 'RETEMPN', 'MWTEMPN']

In [ ]:
['year','transit','modelrunID','tothh_share'  ,'hhq1_share'  ,'totemp_share' ,'RETEMPN_share','MWTEMPN_share','modelrunID']

        prox_sdf_taz['tothh_share'  ] = round(prox_sdf_taz.tothh  / prox_sdf_taz.tothh.sum(),  2)
        prox_sdf_taz['hhq1_share'   ] = round(prox_sdf_taz.hhq1   / prox_sdf_taz.hhq1.sum()  , 2)
        prox_sdf_taz['totemp_share' ] = round(prox_sdf_taz.totemp / prox_sdf_taz.totemp.sum(), 2)
        prox_sdf_taz['RETEMPN_share'] = round(prox_sdf_taz.RETEMPN/prox_sdf_taz.RETEMPN.sum(), 2)
        prox_sdf_taz['MWTEMPN_share'] = round(prox_sdf_taz.MWTEMPN/prox_sdf_taz.MWTEMPN.sum(), 2)
        prox_sdf_taz['year'] = str(model_year)
        prox_sdf_taz['modelrunID'] = us_runid
        prox_sdf_taz['transit'] = transit_feature

# Sandbox

In [468]:
def create_multiple_transit_areas_old(transit_stop_gdf, scenario='current', write_to_disk=False):

    logger.info(
        f"Running create_transit_areas function for variant: {scenario}")
    logger.info(f"Transit Stop input geodataframe has shape: {transit_stop_gdf.shape}")

    buf_dist_half = .5

    qry_str_major = "(major_stop == 1)"
    logger.info(
        f"Subsetting stops to {qry_str_major}, and adding {buf_dist_half} mile buffer")
    major_buf = station_buffer(
        transit_stop_gdf, qry_str_major, buf_dist_half)

    buf_dist_qtr = .25

    qry_str_hdwy_lt15 = "(am_av_hdwy <= 15) & (pm_av_hdwy <= 15)"
    logger.info(
        f"Subsetting stops to {qry_str_hdwy_lt15}, and adding {buf_dist_qtr} mile buffer")
    hdwy15_buf = station_buffer(
        transit_stop_gdf, qry_str_hdwy_lt15, buf_dist_qtr)

    qry_str_hdwy_15_to_30 = "( am_av_hdwy >15 ) & (am_av_hdwy <=30) & ( pm_av_hdwy >15 ) & (pm_av_hdwy <=30)  "
    logger.info(
        f"Subsetting stops to {qry_str_hdwy_15_to_30}, and adding {buf_dist_qtr} mile buffer")
    hdwy30_buf = station_buffer(
        transit_stop_gdf, qry_str_hdwy_15_to_30, buf_dist_qtr)

    qry_str_hdwy_gt30 = "(am_av_hdwy > 30) | (pm_av_hdwy > 30)"
    logger.info(
        f"Subsetting stops to {qry_str_hdwy_gt30}, and adding {buf_dist_qtr} mile buffer")
    hdwy30plus_buf = station_buffer(
        transit_stop_gdf, qry_str_hdwy_gt30, buf_dist_qtr)

    ## Union and classify

    # these are the frames we are interested in for the union. We will denote in the
    # union process whether a geometry is covered by a particular spatial frame

    # the ordering here matters - reflecting a hierarchy of interest: we will use
    # to classify areas based on the highest order category in this list
    source_frames_ordered = ['major_buf', 'hdwy15buf', 'hdwy30buf', 'hdwy30plusbuf',
                             'reg']

    # geopandas can only handle two input layers at a time when unioning. We do it step-wise.

    logger.info(
        f"Beginning a series of unions, to get all buffers to fully define regional space")

    logger.info(f"Unioning hdwy15_buf and hdwy30_buf")

    # row_ids are in the source frames; we disambiguate to denote whether a particular
    # area is "covered" by either of the source files.
    # As we are dealing with dissolved geometries for the source geometries, it means
    # each source DF has just 1 row with multipart geoms - and that row_id then just
    # has the index value of 0 carried forward. So in the resulting union, a row_id value
    # of 0 means that a given area was represented in the relevant source dataframe;
    # a value of NaN means there was no corresponding match for a particular dataframe
    # this is useful for classifying areas based on the "source" buffer geometries present
    # for a particular location

    union_1 = gpd.overlay(hdwy15_buf, hdwy30_buf, how='union').reset_index().rename(
        columns={'index': 'df_1', 'rowid_1': 'hdwy15buf', 'rowid_2': 'hdwy30buf'})

    logger.info(f'union_1 shape: {union_1.shape}')

    logger.info(f"Adding hdwy30plus_buf...")
    union_2 = gpd.overlay(union_1, hdwy30plus_buf, how='union').reset_index().rename(
        columns={'index': 'df_2', 'rowid': 'hdwy30plusbuf'})

    # return union_1,hdwy15_buf, hdwy30_buf,union_2,hdwy30plus_buf

    logger.info(f'union_2 shape: {union_2.shape}')

    logger.info(f"Adding major stops buffer...")

    union_3 = gpd.overlay(union_2, major_buf, how='union').reset_index(
    ).rename(columns={'index': 'df_3', 'rowid': 'major_buf'})

    logger.info(f'union_3 shape: {union_3.shape}')
    logger.info(
        f"Adding regional boundary so we can get wall to wall coverage")

    # added small (here 10cm) geometry simplification to avoid a TopologyException: found non-noded intersection
    union_3['geometry'] = union_3.simplify(.1)

    union_4 = gpd.overlay(union_3, region_bdry, how='union').reset_index(
    ).rename(columns={'index': 'df_4', 'name': 'reg'})
    logger.info(f'union_4 shape: {union_4.shape}')
    logger.info('-'*80)
    logger.info(f'union_4 head: {union_4.head().T}')
    logger.info('-'*80)

    # except:
#     logger.error("Traceback info:\n{}\nError Info:\n{}".format(
#         'unions', str(sys.exc_info()[1])))

    # fill nas with -1 - those are values where a particular transit buffer layer doesn't cover something
    union_4 = union_4.fillna(-1)

    # we need boolean values to grab the group names
    logger.info(
        f"With the full region unioned, we now focus on classifying the different combinations")
    union_4[source_frames_ordered] = union_4[source_frames_ordered].replace(
        {0: True, -1: False})

    # get an integer encoding the unique combination of union components.
    # e.g. some areas will be in the buffer of a major stop and a 15 min headway
    # but not a 30 min headway. Others will only be in a 15 min headway buffer
    # we group areas that share the same combination. We then use that in the
    # naming of those groups just downstream; naming them based on
    # the boolean True values (indicating a particular buffer is represented)

    union_4['ngroup'] = union_4.groupby(source_frames_ordered).ngroup()

    # Then, for each combination of overlapping areas, get the most important
    # where rail (major stop) trumps 15min headway; 15min headway trumps 30 min headway etc

    union_4['cat5'] = union_4.ngroup.map(union_4.groupby(['ngroup'])[source_frames_ordered].apply(
        get_combination_names_hierarchy))

    # get the full string list of components in any given unioned area
    union_4['ngroup_desc'] = union_4.ngroup.map(union_4.groupby(['ngroup'])[source_frames_ordered].apply(
        get_combination_names))

    if write_to_disk:
        union_4.to_file(f'/Users/aolsen/Downloads/union_4_{scenario}.geojson',driver='GeoJSON')

    return union_4, union_4.dissolve('cat5', as_index=False)[['cat5', 'geometry']]

In [469]:
list_us_runid

['EIR runs/Baseline Large (s25) runs/NP_v8_FINAL/run314',
 'EIR runs/Alt1 (s26) runs/Alt1_v3_test_far_tiers_FINAL_EIR_ALT/run375',
 'EIR runs/Alt2 (s28) runs/Alt2_v1_FINAL_EIR_ALT/run374',
 'Final Blueprint runs/Final Blueprint (s24)/BAUS v2.25 - FINAL VERSION/run182']

In [470]:
# Bring in urbansim results
all_prox = pd.DataFrame()
taz_prox = pd.DataFrame()

for us_runid in list_us_runid:
    logger.info("")
    logger.info("==== Processing UrbanSim run {} ====".format(us_runid))
    urbansim_runid = Path(urbansim_run_location, us_runid)

    for model_year in (2015, 2050):
        if model_year == 2050:
            if us_runid == us_2050_FBP_Final:
                parcel_file = urbansim_runid + \
                    '_parcel_data_{}_UBI.csv'.format(model_year)
            else:
                parcel_file = urbansim_runid + \
                    '_parcel_data_{}.csv'.format(model_year)
        else:
            parcel_file = urbansim_runid + \
                '_parcel_data_{}.csv'.format(model_year)
        logger.info('Reading {} parcel data from {}'.format(
            model_year, parcel_file))
        parcel_output = pd.read_csv(parcel_file, engine='python')

        logger.info('  Read {} rows'.format(len(parcel_output)))

        # keep essential columns
        parcel_output.drop(['geom_id', 'total_job_spaces', 'zoned_du',
                            'zoned_du_underbuild', 'zoned_du_underbuild_nodev', 'first_building_type'], axis=1, inplace=True)
        logger.info("Head:\n{}".format(parcel_output.head()))

        parcel_output['totemp'] = parcel_output['totemp'].fillna(0)
        parcel_output['totemp'] = parcel_output['totemp'].round(
            0).astype('int')
        parcel_output['RETEMPN'] = parcel_output['RETEMPN'].fillna(0)
        parcel_output['RETEMPN'] = parcel_output['RETEMPN'].round(
            0).astype('int')
        parcel_output['MWTEMPN'] = parcel_output['MWTEMPN'].fillna(0)
        parcel_output['MWTEMPN'] = parcel_output['MWTEMPN'].round(
            0).astype('int')

        # save as table in gdb
#         parcel_table = Path(arcpy.env.workspace, "parcel_table")
#         if arcpy.Exists(parcel_table): arcpy.management.Delete(parcel_table)

#         parcel_array = np.array(np.rec.fromrecords(parcel_output.values))
#         parcel_array.dtype.names = tuple(parcel_output.dtypes.index.tolist())
#         arcpy.da.NumPyArrayToTable(parcel_array, parcel_table)
#         logger.info("Saved to {} in {}".format(parcel_table, arcpy.env.workspace))

        # convert to point feature class in GDB
#         if arcpy.Exists('parcel_fc'): arcpy.management.Delete('parcel_fc')
#         arcpy.management.XYTableToPoint(in_table=parcel_table, out_feature_class='parcel_fc',x_field='x',y_field='y')
#         logger.info("Saved to {} in {}".format('parcel_fc', arcpy.env.workspace))

03/21/2024 03:57:09 PM - INFO - 
03/21/2024 03:57:09 PM - INFO - ==== Processing UrbanSim run EIR runs/Baseline Large (s25) runs/NP_v8_FINAL/run314 ====
03/21/2024 03:57:09 PM - INFO - Reading 2015 parcel data from /Users/aolsen/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/EIR runs/Baseline Large (s25) runs/NP_v8_FINAL/run314_parcel_data_2015.csv


KeyboardInterrupt: 

In [ ]:
if model_year == 2015:
        # current
        transit_features = ['trn_cur_cat5']
    elif model_year == 2050:
        # no plan and blueprint
        transit_features = ['trn_np_cat5', 'trn_fp_cat5']

    for transit_feature in transit_features:

        logger.info('Summarizing {} parcel data proximity to {}'.format(
            model_year, transit_feature))
        log_workspace_contents(logger)

        try:
            logger.info("feature classes no paths")
            arcpy.SummarizeWithin_analysis(transit_feature, 'parcel_fc', 'prox', keep_all_polygons="KEEP_ALL",
                                            sum_fields=[['tothh', 'SUM'], ['hhq1','SUM'],['totemp','SUM'],['RETEMPN','SUM'],['MWTEMPN','SUM']])
            # hasn't worked, see comments below
            logger.info("SUCCESS")
        except:
            # Get the tool error messages
            msgs = arcpy.GetMessages(2)
            logger.error("Exception occured; msgs: {}".format(msgs))

            # Get the traceback object
            tb = sys.exc_info()[2]
            tbinfo = traceback.format_tb(tb)[0]

            # Concatenate information together concerning the error into a message string
            logger.error("Traceback info:\n{}\nError Info:\n{}".format(
                tbinfo, str(sys.exc_info()[1])))
            logger.error(
                "It's ok though -- we'll do this another way, but still trying the easy way")

        # Something related to arcpy.SummarizeWithin_analysis() is buggy
        # The following attempts have failed with
        # ERROR 000187: Only supports Geodatabase tables and feature classes
        # * use the method with feature layers as inputs, with full paths and without
        # * use the method with feature classes as inputs, with full paths and without
        # * call the short script below via subprocess
        # * copy feature classes to arcpy.env.scratchGDB and summarizeWithin there
        #
        # HOWEVER, after this script fails, running the following on the command line succeeds:
        #
        # >>> import arcpy
        # >>> arcpy.env.workspace='M:\Data\GIS layers\JobsHousingTransitProximity\workspace_2020_1007_1737.gdb'
        # >>> arcpy.SummarizeWithin_analysis('trn_cur_cat5', 'parcel_fc', 'prox', keep_all_polygons='KEEP_ALL', sum_fields=[['tothh','SUM'], ['hhq1','SUM'],['totemp','SUM'],['RETEMPN','SUM'],['MWTEMPN','SUM']])
        # <Result 'M:\\Data\\GIS layers\\JobsHousingTransitProximity\\workspace_2020_1007_1737.gdb\\prox'>
        # >>> prox_sdf = pd.DataFrame.spatial.from_featureclass('prox')
        #
        # so we'll summarize within ourselves and use spatial join instead
        if arcpy.Exists('parcel_fc_join_trn'):
            arcpy.management.Delete('parcel_fc_join_trn')
        logger.info("spatial joining parcel_fc with {}".format(transit_feature))
        arcpy.SpatialJoin_analysis(target_features='parcel_fc', join_features=transit_feature,
                                   out_feature_class='parcel_fc_join_trn')

        logger.info("spatial joining parcel_fc_join_trn with {}".format(taz))
        arcpy.SpatialJoin_analysis(target_features='parcel_fc_join_trn', join_features=taz,
                                   out_feature_class='parcel_fc_join_trn_taz')

        logger.info(
            "spatial joining parcel_fc_join_trn_taz with {}".format(tract))
        arcpy.SpatialJoin_analysis(target_features='parcel_fc_join_trn_taz', join_features=tract,
                                   out_feature_class='parcel_fc_join_trn_taz_tract')
        logger.info("    ...complete")

        prox_sdf_ba = pd.DataFrame.spatial.from_featureclass(
            'parcel_fc_join_trn_taz_tract')
        prox_sdf = prox_sdf_ba.groupby('Service_Level').agg({'tothh': 'sum', 'hhq1':'sum', 
                                                         'totemp': 'sum', 'RETEMPN':'sum',
                                                         'MWTEMPN': 'sum'}).reset_index()

        prox_sdf['tothh_share'  ] = round(prox_sdf.tothh  / prox_sdf.tothh.sum(),  2)
        prox_sdf['hhq1_share'   ] = round(prox_sdf.hhq1   / prox_sdf.hhq1.sum()  , 2)
        prox_sdf['totemp_share' ] = round(prox_sdf.totemp / prox_sdf.totemp.sum(), 2)
        prox_sdf['RETEMPN_share'] = round(prox_sdf.RETEMPN/prox_sdf.RETEMPN.sum(), 2)
        prox_sdf['MWTEMPN_share'] = round(prox_sdf.MWTEMPN/prox_sdf.MWTEMPN.sum(), 2)
        prox_sdf['year'] = str(model_year)
        prox_sdf['modelrunID'] = us_runid
        prox_sdf['transit'] = transit_feature
        prox_sdf['area'] = 'Region'

        logger.info("prox_sdf:\n{}".format(prox_sdf))
        all_prox = all_prox.append(prox_sdf)

        prox_sdf_epc = prox_sdf_ba[prox_sdf_ba.epc == 1]
        prox_sdf_epc = prox_sdf_epc.groupby('Service_Level').agg({'tothh': 'sum', 'hhq1':'sum', 
                                                         'totemp': 'sum', 'RETEMPN':'sum',
                                                         'MWTEMPN': 'sum'}).reset_index()

        prox_sdf_epc['tothh_share'  ] = round(prox_sdf_epc.tothh  / prox_sdf_epc.tothh.sum(),  2)
        prox_sdf_epc['hhq1_share'   ] = round(prox_sdf_epc.hhq1   / prox_sdf_epc.hhq1.sum()  , 2)
        prox_sdf_epc['totemp_share' ] = round(prox_sdf_epc.totemp / prox_sdf_epc.totemp.sum(), 2)
        prox_sdf_epc['RETEMPN_share'] = round(prox_sdf_epc.RETEMPN/prox_sdf_epc.RETEMPN.sum(), 2)
        prox_sdf_epc['MWTEMPN_share'] = round(prox_sdf_epc.MWTEMPN/prox_sdf_epc.MWTEMPN.sum(), 2)
        prox_sdf_epc['year'] = str(model_year)
        prox_sdf_epc['modelrunID'] = us_runid
        prox_sdf_epc['transit'] = transit_feature
        prox_sdf_epc['area'] = 'epcs'

        logger.info("prox_sdf_epc:\n{}".format(prox_sdf_epc))
        all_prox = all_prox.append(prox_sdf_epc)

        prox_sdf_hra = prox_sdf_ba[prox_sdf_ba.hra == 1]
        prox_sdf_hra = prox_sdf_hra.groupby('Service_Level').agg({'tothh': 'sum', 'hhq1':'sum', 
                                                         'totemp': 'sum', 'RETEMPN':'sum',
                                                         'MWTEMPN': 'sum'}).reset_index()

        prox_sdf_hra['tothh_share'  ] = round(prox_sdf_hra.tothh  / prox_sdf_hra.tothh.sum(),  2)
        prox_sdf_hra['hhq1_share'   ] = round(prox_sdf_hra.hhq1   / prox_sdf_hra.hhq1.sum()  , 2)
        prox_sdf_hra['totemp_share' ] = round(prox_sdf_hra.totemp / prox_sdf_hra.totemp.sum(), 2)
        prox_sdf_hra['RETEMPN_share'] = round(prox_sdf_hra.RETEMPN/prox_sdf_hra.RETEMPN.sum(), 2)
        prox_sdf_hra['MWTEMPN_share'] = round(prox_sdf_hra.MWTEMPN/prox_sdf_hra.MWTEMPN.sum(), 2)
        prox_sdf_hra['year'] = str(model_year)
        prox_sdf_hra['modelrunID'] = us_runid
        prox_sdf_hra['transit'] = transit_feature
        prox_sdf_hra['area'] = 'HRAs'

        logger.info("prox_sdf_hra:\n{}".format(prox_sdf_hra))
        all_prox = all_prox.append(prox_sdf_hra)

        logger.info("all_prox:\n{}".format(all_prox))

        prox_sdf_taz = prox_sdf_ba.groupby(['area_type', 'Service_Level']).agg({'tothh':'sum', 'hhq1':'sum', 
                                                         'totemp': 'sum', 'RETEMPN':'sum',
                                                         'MWTEMPN': 'sum'}).reset_index()

        prox_sdf_taz['tothh_share'  ] = round(prox_sdf_taz.tothh  / prox_sdf_taz.tothh.sum(),  2)
        prox_sdf_taz['hhq1_share'   ] = round(prox_sdf_taz.hhq1   / prox_sdf_taz.hhq1.sum()  , 2)
        prox_sdf_taz['totemp_share' ] = round(prox_sdf_taz.totemp / prox_sdf_taz.totemp.sum(), 2)
        prox_sdf_taz['RETEMPN_share'] = round(prox_sdf_taz.RETEMPN/prox_sdf_taz.RETEMPN.sum(), 2)
        prox_sdf_taz['MWTEMPN_share'] = round(prox_sdf_taz.MWTEMPN/prox_sdf_taz.MWTEMPN.sum(), 2)
        prox_sdf_taz['year'] = str(model_year)
        prox_sdf_taz['modelrunID'] = us_runid
        prox_sdf_taz['transit'] = transit_feature

        logger.info("prox_sdf:\n{}".format(prox_sdf_taz))
        taz_prox = taz_prox.append(prox_sdf_taz)

        logger.info("taz_prox:\n{}".format(taz_prox))

In [269]:
# union_1.to_file('/Users/aolsen/Downloads/union_1.geojson', driver='GeoJSON')
# union_2.to_file('/Users/aolsen/Downloads/union_2.geojson', driver='GeoJSON')
# union_3.to_file('/Users/aolsen/Downloads/union_3.geojson', driver='GeoJSON')
# union_4.to_file('/Users/aolsen/Downloads/union_4.geojson', driver='GeoJSON')

In [236]:
# hdwy15_buf.to_file(
#     '/Users/aolsen/Downloads/hdwy15_buf.geojson', driver='GeoJSON')
# hdwy30_buf.to_file(
#     '/Users/aolsen/Downloads/hdwy30_buf.geojson', driver='GeoJSON')
# hdwy30plus_buf.to_file(
#     '/Users/aolsen/Downloads/hdwy30plus_buf.geojson', driver='GeoJSON')
# major_buf.to_file(
#     '/Users/aolsen/Downloads/major_buf.geojson', driver='GeoJSON')

In [237]:

# diff_1 = gpd.overlay(hdwy15_buf,major_buf,how='difference')

# diff_2 = gpd.overlay(hdwy30_buf,major_buf,how='difference')
# diff_2b = gpd.overlay(diff_2,hdwy15_buf,how='difference')

# diff_3 = gpd.overlay(hdwy30plus_buf,major_buf,how='difference')
# diff_3b = gpd.overlay(diff_3,hdwy15_buf,how='difference')
# diff_3c = gpd.overlay(diff_3b,hdwy30_buf,how='difference')

In [ ]:
# check prefix+'_cat5'

In [ ]:
    logger.info("==== create_transit_features({}) ====".format(transit_type))
    if transit_type == "current":
        input_layer = "transit_current"
        prefix = "trn_cur"
    elif transit_type == "noplan":
        input_layer = "transit_potential"
        prefix = "trn_np"
        curprefix = "trn_cur"
    elif transit_type == "blueprint":
        input_layer = "transit_potential"
        prefix = "trn_fp"
        curprefix = "trn_cur"
    else:
        logger.fatal("Unsupported transit_type {}".format(transit_type))

In [113]:
transit_type_config = {
    "current": {
        "input_layer": "transit_current",
        "prefix": "trn_cur",
    },
    "noplan": {
        "input_layer": "transit_potential",
        "prefix": "trn_np",
        "curprefix": "trn_cur",
    },
    "blueprint": {
        "input_layer": "transit_potential",
        "prefix": "trn_fp",
        "curprefix": "trn_cur",
    },
}


In [7]:
def create_transit_features_new(logger, transit_type):
    """
    transit_type is one of [current, noplan, blueprint]
    """


    orig_result = arcpy.GetCount_management(input_layer)
    logger.info("{} has {} rows".format(input_layer, orig_result[0]))
    
    
    if transit_type == "current":
        logger.info('Select buffer for major transit stops => {}_majorbuf'.format(prefix))
        major = arcpy.SelectLayerByAttribute_management(input_layer, "NEW_SELECTION", '"major_stop" = 1')
        arcpy.CopyFeatures_management(major, prefix+"_major")  # save selection to new feature class
        major_result = arcpy.GetCount_management(prefix+"_major")
        logger.info("  {}_major has {} rows".format(prefix, major_result[0]))
        arcpy.Buffer_analysis(prefix+"_major", prefix+"_majorbuf", "0.5 Miles", "FULL", "ROUND", "ALL", None, "PLANAR")
   
    else:
        if transit_type=="noplan":
            logger.info('Selecting "Under Construction" or "Open" stops for no plan')
            new_major = arcpy.management.SelectLayerByAttribute(input_layer, "NEW_SELECTION", 
                                                                '"status" = \'Under Construction\' Or status=\'Open\'')
        elif transit_type=="blueprint":
            logger.info('Selecting "Under Construction" or "Open" or "Final Blueprint" stops for no plan')
            new_major = arcpy.management.SelectLayerByAttribute(input_layer, "NEW_SELECTION", 
                                                                '"status" = \'Under Construction\' Or status=\'Open\' Or status=\'Final Blueprint\'')

        arcpy.CopyFeatures_management(new_major, prefix+"_new_major")  # save selection to new feature class
        new_major_result = arcpy.GetCount_management(prefix+"_new_major")
        logger.info("  {}_new_major has {} rows".format(prefix, new_major_result[0]))
        arcpy.Buffer_analysis(prefix+"_new_major", prefix+"_newmajorbuf", "0.5 Miles", "FULL", "ROUND", "ALL", None, "PLANAR")

        # merge new major buffered with original major buffered
        arcpy.Merge_management([curprefix+"_majorbuf",prefix+"_newmajorbuf"], prefix+"_majorbuf_predissolve")
        # dissolve
        arcpy.management.Dissolve(prefix+"_majorbuf_predissolve", prefix+"_majorbuf", 
                                  dissolve_field=None, statistics_fields=None, multi_part="MULTI_PART", unsplit_lines="DISSOLVE_LINES")


In [ ]:
def create_transit_features(logger, transit_type):
    """
    transit_type is one of [current, noplan, blueprint]
    """
    logger.info("==== create_transit_features({}) ====".format(transit_type))
    if transit_type == "current":
        input_layer = "transit_current"
        prefix = "trn_cur"
    elif transit_type == "noplan":
        input_layer = "transit_potential"
        prefix = "trn_np"
        curprefix = "trn_cur"
    elif transit_type == "blueprint":
        input_layer = "transit_potential"
        prefix = "trn_fp"
        curprefix = "trn_cur"
    else:
        logger.fatal("Unsupported transit_type {}".format(transit_type))

    orig_result = arcpy.GetCount_management(input_layer)
    logger.info("{} has {} rows".format(input_layer, orig_result[0]))

    # buffered transit_current major stops
    if transit_type == "current":
        logger.info(
            'Select buffer for major transit stops => {}_majorbuf'.format(prefix))
        major = arcpy.SelectLayerByAttribute_management(
            input_layer, "NEW_SELECTION", '"major_stop" = 1')
        # save selection to new feature class
        arcpy.CopyFeatures_management(major, prefix+"_major")
        major_result = arcpy.GetCount_management(prefix+"_major")
        logger.info("  {}_major has {} rows".format(prefix, major_result[0]))
        arcpy.Buffer_analysis(prefix+"_major", prefix+"_majorbuf",
                              "0.5 Miles", "FULL", "ROUND", "ALL", None, "PLANAR")

    else:
        if transit_type == "noplan":
            logger.info(
                'Selecting "Under Construction" or "Open" stops for no plan')
            new_major = arcpy.management.SelectLayerByAttribute(input_layer, "NEW_SELECTION",
                                                                '"status" = \'Under Construction\' Or status=\'Open\'')
        elif transit_type == "blueprint":
            logger.info(
                'Selecting "Under Construction" or "Open" or "Final Blueprint" stops for no plan')
            new_major = arcpy.management.SelectLayerByAttribute(input_layer, "NEW_SELECTION",
                                                                '"status" = \'Under Construction\' Or status=\'Open\' Or status=\'Final Blueprint\'')

        # save selection to new feature class
        arcpy.CopyFeatures_management(new_major, prefix+"_new_major")
        new_major_result = arcpy.GetCount_management(prefix+"_new_major")
        logger.info("  {}_new_major has {} rows".format(
            prefix, new_major_result[0]))
        arcpy.Buffer_analysis(prefix+"_new_major", prefix+"_newmajorbuf",
                              "0.5 Miles", "FULL", "ROUND", "ALL", None, "PLANAR")

        # merge new major buffered with original major buffered
        arcpy.Merge_management(
            [curprefix+"_majorbuf", prefix+"_newmajorbuf"], prefix+"_majorbuf_predissolve")
        # dissolve
        arcpy.management.Dissolve(prefix+"_majorbuf_predissolve", prefix+"_majorbuf",
                                  dissolve_field=None, statistics_fields=None, multi_part="MULTI_PART", unsplit_lines="DISSOLVE_LINES")

    # buffered hdway_15min stops
    logger.info(
        'Creating buffer for stops with headway < 15min => {}_hdwy15buf'.format(prefix))

    if transit_type == "current":
        hdwy15 = arcpy.SelectLayerByAttribute_management(
            input_layer, "NEW_SELECTION", "am_av_hdwy <= 15 And pm_av_hdwy <= 15")
        # save selection to new feature class
        arcpy.CopyFeatures_management(hdwy15, prefix+"_hdwy15")
        hdwy15_result = arcpy.GetCount_management(prefix+"_hdwy15")
        logger.info("  {}_hdwy15 has {} rows".format(prefix, hdwy15_result[0]))
        arcpy.Buffer_analysis(prefix+"_hdwy15", prefix+"_hdwy15buf",
                              "0.25 Miles", "FULL", "ROUND", "ALL", None, "PLANAR")

    else:
        # copy current
        arcpy.CopyFeatures_management(
            curprefix+"_hdwy15buf", prefix+"_hdwy15buf")

    # buffered hdway_30min stops
    logger.info(
        'Creating buffer for stops with headway 15-30 min => {}_hdwy30buf'.format(prefix))

    if transit_type == "current":
        hdwy30 = arcpy.SelectLayerByAttribute_management(
            input_layer, "NEW_SELECTION", "am_av_hdwy > 15 And am_av_hdwy <= 30 And pm_av_hdwy > 15 And pm_av_hdwy <= 30")
        # save selection to new feature class
        arcpy.CopyFeatures_management(hdwy30, prefix+"_hdwy30")
        hdwy30_result = arcpy.GetCount_management(prefix+"_hdwy30")
        logger.info("  {}_hdwy30 has {} rows".format(prefix, hdwy30_result[0]))
        arcpy.Buffer_analysis(prefix+"_hdwy30", prefix+"_hdwy30buf",
                              "0.25 Miles", "FULL", "ROUND", "ALL", None, "PLANAR")

    else:
        # copy current
        arcpy.CopyFeatures_management(
            curprefix+"_hdwy30buf", prefix+"_hdwy30buf")

    # buffered hdway_30plus stops
    logger.info(
        'Creating buffer for stops with headway 30+ min => {}_hdwy30plusbuf'.format(prefix))

    if transit_type == "current":
        hdwy30plus = arcpy.SelectLayerByAttribute_management(
            input_layer, "NEW_SELECTION", "am_av_hdwy > 30 Or pm_av_hdwy > 30")
        # save selection to new feature class
        arcpy.CopyFeatures_management(hdwy30plus, prefix+"_hdwy30plus")
        hdwy30plus_result = arcpy.GetCount_management(prefix+"_hdwy30plus")
        logger.info("  {}_hdwy30plus has {} rows".format(
            prefix, hdwy30plus_result[0]))
        arcpy.Buffer_analysis(prefix+"_hdwy30plus", prefix+"_hdwy30plusbuf",
                              "0.25 Miles", "FULL", "ROUND", "ALL", None, "PLANAR")

    else:
        # copy current
        arcpy.CopyFeatures_management(
            curprefix+"_hdwy30plusbuf", prefix+"_hdwy30plusbuf")

    # Make them disjoint -- first one wins
    logger.info(
        'Isolate {}_hdwy15buf => {}_hdwy15buf_only'.format(prefix, prefix))
    arcpy.Erase_analysis(in_features=prefix+"_hdwy15buf", erase_features=prefix+"_majorbuf",
                         out_feature_class=prefix+"_hdwy15buf_only")

    logger.info(
        'Isolate {}_hdwy30buf => {}_hdwy30buf_only'.format(prefix, prefix))
    arcpy.Erase_analysis(prefix+"_hdwy30buf",   prefix +
                         "_majorbuf",  prefix+"_hdwy30buf_1")
    arcpy.Erase_analysis(prefix+"_hdwy30buf_1", prefix +
                         "_hdwy15buf", prefix+"_hdwy30buf_only")
    arcpy.Delete_management([prefix+"_hdwy30buf_1"])

    logger.info(
        'Isolate {}_hdwy30plusbuf  => {}_hdwy30plusbuf_only'.format(prefix, prefix))
    arcpy.Erase_analysis(prefix+"_hdwy30plusbuf",   prefix +
                         "_majorbuf",  prefix+"_hdwy30plusbuf_1")
    arcpy.Erase_analysis(prefix+"_hdwy30plusbuf_1", prefix +
                         "_hdwy15buf", prefix+"_hdwy30plusbuf_2")
    arcpy.Erase_analysis(prefix+"_hdwy30plusbuf_2", prefix +
                         "_hdwy30buf", prefix+"_hdwy30plusbuf_only")
    arcpy.Delete_management([prefix+"_hdwy30plusbuf_1",
                             prefix+"_hdwy30plusbuf_2"])

    logger.info('Rest of Bay Area => {}_none'.format(prefix))
    arcpy.Erase_analysis("BAcounty_expand",   prefix +
                         "_majorbuf",      "BAcounty_expand_1")
    arcpy.Erase_analysis("BAcounty_expand_1", prefix +
                         "_hdwy15buf",     "BAcounty_expand_2")
    arcpy.Erase_analysis("BAcounty_expand_2", prefix +
                         "_hdwy30buf",     "BAcounty_expand_3")
    arcpy.Erase_analysis("BAcounty_expand_3", prefix +
                         "_hdwy30plusbuf", prefix+"_none")
    arcpy.Delete_management(["BAcounty_expand_1",
                             "BAcounty_expand_2",
                             "BAcounty_expand_3"])

    logger.info('Merge into one feature class => {}_cat5'.format(prefix))
    arcpy.Merge_management([prefix+"_none",
                            prefix+"_hdwy30plusbuf_only",
                            prefix+"_hdwy30buf_only",
                            prefix+"_hdwy15buf_only",
                            prefix+"_majorbuf"],
                           prefix+"_cat5", add_source="ADD_SOURCE_INFO")
    # create Service_Level from MERGE_SRC
    arcpy.AddField_management(
        prefix + "_cat5", "Service_Level", "TEXT", "", "", 200)
    with arcpy.da.UpdateCursor(prefix + "_cat5", ["Service_Level", "MERGE_SRC"]) as cursor:
        for row in cursor:
            if 'none' in row[1]:
                row[0] = 'No_Fixed_Route_Transit'
            elif 'hdwy30plusbuf' in row[1]:
                row[0] = 'Bus_31plus_min'
            elif 'hdwy30buf' in row[1]:
                row[0] = 'Bus_15_30min'
            elif 'hdwy15buf' in row[1]:
                row[0] = 'Bus_<15min'
            elif 'majorbuf' in row[1]:
                row[0] = 'Major_Transit_Stop'
            cursor.updateRow(row)

In [6]:
import geopandas as gpd
from shapely.geometry import Point  # for creating point buffers


def create_transit_features(logger, transit_type):
    """
    transit_type is one of [current, noplan, blueprint]
    """
    logger.info("==== create_transit_features({}) ====".format(transit_type))

    # Input data loading (adjust path and format)
    # Replace with correct path and format
    input_gdf = gpd.read_file("path/to/your/data.shp")

    if transit_type == "current":
        prefix = "trn_cur"
        curprefix = "trn_cur"
    elif transit_type == "noplan":
        prefix = "trn_np"
        curprefix = "trn_cur"
    elif transit_type == "blueprint":
        prefix = "trn_fp"
        curprefix = "trn_cur"
    else:
        logger.fatal("Unsupported transit_type {}".format(transit_type))

    # Calculate feature counts (use describe method)
    feature_counts = input_gdf.describe()
    logger.info(f"{input_layer} has {feature_counts['geometry'].iloc[0]} rows")

    # Select features based on conditions (use query method)
    if transit_type == "current":
        major_gdf = input_gdf.query("major_stop == 1")
    else:
        if transit_type == "noplan":
            new_major_gdf = input_gdf.query(
              "(status == 'Under Construction') | (status == 'Open')"
          )
    else:
        new_major_gdf = input_gdf.query(
          "(status == 'Under Construction') | (status == 'Open') | (status == 'Final Blueprint')"
      )

    # Buffer features (use buffer method)
    majorbuf_gdf = major_gdf.buffer(distance=0.5, units="miles")
    if transit_type != "current":
        hdwy15buf_gdf = gpd.read_file(
            f"{curprefix}_hdwy15buf.shp")  # Assuming existing buffer
        hdwy30buf_gdf = gpd.read_file(
            f"{curprefix}_hdwy30buf.shp")  # Assuming existing buffer
        hdwy30plusbuf_gdf = gpd.read_file(
            f"{curprefix}_hdwy30plusbuf.shp")  # Assuming existing buffer

    # ... (similar logic for hdwy15, hdwy30, hdwy30plus selections and buffering using query and buffer methods)

    # Isolate buffers using difference (use difference method)
    if transit_type == "current":
        hdwy15buf_only_gdf = majorbuf_gdf.difference(hdwy30buf_gdf)
        hdwy30buf_only_gdf = (
            majorbuf_gdf.difference(
                hdwy15buf_gdf).difference(hdwy30plusbuf_gdf)
        )
        hdwy30plusbuf_only_gdf = majorbuf_gdf.difference(
            hdwy15buf_gdf).difference(hdwy30buf_gdf)
    else:
    # ... (similar logic for isolating buffers using difference for noplan and blueprint)
        pass

    # Isolate remaining area using difference (use difference method)
    none_gdf = gpd.read_file("BAcounty_expand.shp")  # Assuming existing data
    none_gdf = none_gdf.difference(majorbuf_gdf)
    if transit_type != "current":
        none_gdf = none_gdf.difference(hdwy15buf_gdf)
        none_gdf = none_gdf.difference(hdwy30buf_gdf)
        none_gdf = none_gdf.difference(hdwy30plusbuf_gdf)

    # Merge into one GeoDataFrame (use concat and GeoDataFrame constructor)
    merged_gdf = gpd.GeoDataFrame(pd.concat([
      none_gdf, hdwy30plusbuf_only_gdf, hdwy30buf_only_gdf,
      hdwy15

IndentationError: expected an indented block after 'else' statement on line 34 (3996946805.py, line 35)